In [6]:
# # Remove everything that is not needed (inlcuding parsedData)
# import os
# import shutil

# # List of files/folders to keep
# files_to_keep = [
#     'more files',
#     'catalog breakdown',
#     'Data_Haz.json',
#     'Embeddings',
#     'Pre-process.ipynb',
#     'Chat_bot.ipynb',
#     'New folder'
# ]

# # Function to delete all files/folders except the ones in files_to_keep list
# def delete_files_except_specific():
#     current_directory = os.getcwd()
#     for item in os.listdir(current_directory):
#         item_path = os.path.join(current_directory, item)
#         if item not in files_to_keep:
#             if os.path.isfile(item_path):
#                 os.remove(item_path)
#                 print(f"Deleted file: {item_path}")
#             elif os.path.isdir(item_path):
#                 shutil.rmtree(item_path)
#                 print(f"Deleted folder: {item_path}")

# # Calling the function to delete files/folders
# delete_files_except_specific()

In [7]:
#Remove objects with no description
import json

def remove_empty_descriptions(data):
    if isinstance(data, dict):
        if 'description' in data and data['description'] == "":
            return None  # Return None to indicate that this object should be filtered out
        for key, value in data.items():
            data[key] = remove_empty_descriptions(value)
    elif isinstance(data, list):
        data = [remove_empty_descriptions(item) for item in data if remove_empty_descriptions(item) is not None]
    return data

def process_json_file(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:  # Explicitly specify the encoding
        json_data = json.load(file)

    modified_data = remove_empty_descriptions(json_data)

    with open(output_file_path, 'w', encoding='utf-8') as file:  # Explicitly specify the encoding
        json.dump(modified_data, file, indent=2, ensure_ascii=False)

# Usage example:
input_file_path = 'Data_Haz.json'
output_file_path = 'modified.json'
process_json_file(input_file_path, output_file_path)

In [8]:
# Extract featured courses from the objects and make a file
import json
import os

def extract_featured_courses(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    featured_courses = set(f"Title:{item['name']} Duration:" for item in data)  # Modified this line

    return featured_courses

def write_to_txt_file(file_path, featured_courses):
    with open(file_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write("What are the featured courses in Hazwoper? Tell me the featured courses in Hazwoper. List all the Featured courses. Can you tell me about the featured courses?\n\n")
        for idx, course in enumerate(featured_courses, 1):
            txt_file.write(f"{idx}. {course}\n")
        txt_file.write("####################\n")

def create_and_copy_files(featured_courses):
    folder_path = "catalog breakdown"
    os.makedirs(folder_path, exist_ok=True)

    featured_courses_file = os.path.join(folder_path, "How many featured courses are there.txt")
    write_to_txt_file(featured_courses_file, featured_courses)

if __name__ == "__main__":
    json_file_path = "modified.json"
    featured_courses = extract_featured_courses(json_file_path)

    create_and_copy_files(featured_courses)

In [9]:
#modify json (remove group and name key)
import json

def remove_keys_from_json(data, keys_to_remove):
    for obj in data:
        for key in keys_to_remove:
            obj.pop(key, None)

# Usage example:
if __name__ == "__main__":
    input_file_path = 'modified.json'
    keys_to_remove = ['group', 'name']

    with open(input_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    remove_keys_from_json(data, keys_to_remove)

    with open(input_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, indent=2)

In [10]:
#changing some alt_titles
import json

# Define the mapping of old values to new values
value_mapping = {
    "lithium batteries": "Shipping Lithium Batteries: Fully Regulated Cells and Batteries (U.S. DOT, ICAO/IATA, IMO/IMDG)",
    "lithium cells and batteries": "Lithium Batteries Awareness Training",
    "lithium metal cells": "Shipping Lithium Batteries: Excepted Cells and Batteries (U.S. DOT, ICAO/IATA, IMO/IMDG)",
    "man wearing a facemask traveling using underground rail": "OSHA Bloodborne Pathogens Training",
    "Man with fire extinguisher putting out flames": "OSHA Fire Safety Training",
    "Welding work training as per OSHA Construction Standards": "OSHA Welding, Cutting, and Brazing for Construction Training",
    "DOT, IATA, and IMDG Shipping Lithium Batteries training courses": "Shipping Lithium Batteries: Fully Regulated and Excepted Cells and Batteries Training (U.S. DOT, ICAO/IATA, IMO/IMDG)"
}

# Load the JSON file
with open('modified.json', 'r') as json_file:
    data = json.load(json_file)

# Function to recursively replace values in nested dictionaries
def replace_values(obj):
    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == 'alt_title' and value in value_mapping:
                obj[key] = value_mapping[value]
            elif isinstance(value, (dict, list)):
                replace_values(value)
    elif isinstance(obj, list):
        for item in obj:
            replace_values(item)

# Call the function to replace values
replace_values(data)

# Save the modified data back to the JSON file
with open('modified.json', 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [11]:
#bring hours and price keys right below title
import json

def reorder_keys(json_data):
    new_order = ['title', 'hours', 'price', 'course_category', 'description', 'meta_description', 'Page Type', 'url', 'alt_title']
    for obj in json_data:
        ordered_obj = {key: obj[key] for key in new_order if key in obj}
        for key in obj.keys():
            if key not in new_order:
                ordered_obj[key] = obj[key]
        obj.clear()
        obj.update(ordered_obj)

def main(input_file):
    try:
        with open(input_file, 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        print("Input file not found.")
        return

    reorder_keys(data)

    try:
        with open(input_file, 'w') as f:
            json.dump(data, f, indent=2)
    except:
        print("Error while writing to the input file.")
        return

    print("Processing completed. Data modified in", input_file)

if __name__ == "__main__":
    input_file = "modified.json"  # Replace with the path to your input JSON file
    main(input_file)

Processing completed. Data modified in modified.json


In [12]:
#remove null keys
import json

def remove_null_keys(obj):
    if isinstance(obj, dict):
        return {k: remove_null_keys(v) for k, v in obj.items() if v is not None}
    elif isinstance(obj, list):
        return [remove_null_keys(item) for item in obj]
    else:
        return obj

def process_json_file(input_file):
    with open(input_file, 'r') as f:
        data = json.load(f)

    modified_data = remove_null_keys(data)

    with open(input_file, 'w') as f:
        json.dump(modified_data, f, indent=2)

if __name__ == "__main__":
    input_json_file = "modified.json"

    process_json_file(input_json_file)

In [13]:
#remove alt titles from everywhere except the mentioned keys
import json

def remove_alt_title_except_specific_keys(data):
    if isinstance(data, dict):
        if 'alt_title' in data and data.get('title') not in ['Syllabus', 'Learning Objectives', 'Accreditation & Requirements', 'Overview', 'SCORM Package', 'Group Training']:
            del data['alt_title']
        for key, value in data.items():
            data[key] = remove_alt_title_except_specific_keys(value)
    elif isinstance(data, list):
        data = [remove_alt_title_except_specific_keys(item) for item in data]
    return data

def modify_json_data(data):
    modified_data = remove_alt_title_except_specific_keys(data)
    return modified_data

# Load the JSON data
input_file_path = 'modified.json'
with open(input_file_path, 'r') as file:
    json_data = json.load(file)

# Modify the JSON data in-place
modified_data = modify_json_data(json_data)

# Now the variable modified_data holds the modified JSON structure

# If you want to save the changes back to the same file (optional):
with open(input_file_path, 'w') as file:
    json.dump(modified_data, file, indent=2)

In [14]:
#bring Page Type key in the bottom
import json

def bring_page_type_to_last(data):
    for obj in data:
        if 'Page Type' in obj:
            page_type_value = obj.pop('Page Type')
            obj['Page Type'] = page_type_value
    return data

# Usage example:
if __name__ == "__main__":
    input_file_path = 'modified.json'

    with open(input_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    modified_data = bring_page_type_to_last(data)

    # The data is modified in memory, without creating an output file
    # You can print the modified_data if you want to see the changes
    print(modified_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
#bring alt_title at the bottom for the given titles
import json

def move_alt_title_to_bottom(data):
    if 'alt_title' in data:
        data['alt_title'] = data.pop('alt_title')
    return data

def process_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    for obj in data:
        if 'title' in obj and obj['title'] in ['Syllabus', 'Learning Objectives', 'Accreditation & Requirements', 'Overview','SCORM Package','Group Training']:
            obj = move_alt_title_to_bottom(obj)

    with open(json_file, 'w') as f:
        json.dump(data, f, indent=4)

if __name__ == "__main__":
    json_file_path = 'modified.json'  # Replace with the path to your JSON file
    process_json(json_file_path)

In [16]:
#extract course title from links if alt_title key empty
import json
import re

def extract_name_from_url(url):
    # Use regular expression to extract the name between last two '/'
    match = re.search(r'\/([^\/]+)\/[^\/]+$', url)
    if match:
        return match.group(1)
    return None

def process_json(json_file):
    with open(json_file, 'r+') as f:
        data = json.load(f)

        for obj in data:
            if 'alt_title' in obj and obj['alt_title'] == '':
                url = obj.get('url', '')
                new_alt_title = extract_name_from_url(url)
                if new_alt_title:
                    obj['alt_title'] = new_alt_title

        f.seek(0)  # Move the file pointer to the beginning
        json.dump(data, f, indent=4)
        f.truncate()  # Truncate any remaining content after writing

if __name__ == "__main__":
    input_json_file_path = 'modified.json'  # Replace with the path to your input JSON file
    process_json(input_json_file_path) 

In [17]:
#add $ before price key value
import json

def process_json(json_file):
    with open(json_file, 'r+') as f:
        data = json.load(f)

        for obj in data:
            if 'price' in obj:
                obj['price'] = '$' + str(obj['price'])

        # Move the file pointer to the beginning and truncate the file
        f.seek(0)
        f.truncate()

        # Write the modified data back to the same file
        json.dump(data, f, indent=4)
        f.flush()

if __name__ == "__main__":
    input_json_file_path = 'modified.json'  # Replace with the path to your input JSON file
    process_json(input_json_file_path)

In [18]:
#create q/a files
import os
import json

def create_qna_files(data):
    qna_folder = "qna"
    
    if not os.path.exists(qna_folder):
        os.mkdir(qna_folder)

    course_qna = {}
    
    for item in data:
        course_id = item.get("course_id")
        question = item.get("question")
        answer = item.get("answer")
        alt_title = item.get("alt_title")
        price = item.get("price")
        hours = item.get("hours")
        
        if course_id:
            if course_id not in course_qna:
                course_qna[course_id] = set()
            course_qna[course_id].add((question, answer, alt_title, price, hours))
    
    for course_id, qna_set in course_qna.items():
        file_path = os.path.join(qna_folder, f"{course_id}.txt")
        
        with open(file_path, "w", encoding="utf-8") as f:  # Specify UTF-8 encoding
            for qna in qna_set:
                question, answer, alt_title, price, hours = qna
                f.write(f"Question: {question}\n")
                f.write(f"Answer: {answer}\n")
                f.write(f"Alternative Title: {alt_title}\n")
                f.write(f"Price: {price}\n")
                f.write(f"Hours: {hours}\n")
                f.write("\n")
    
    print("Q&A files created successfully!")

if __name__ == "__main__":
    json_file_path = "modified.json"  # Replace with the path to your JSON file
    
    with open(json_file_path, "r", encoding="utf-8") as json_file:  # Specify UTF-8 encoding
        data = json.load(json_file)
        create_qna_files(data)

Q&A files created successfully!


In [19]:
#remove qna related keys
import json

def remove_keys(data):
    keys_to_remove = ["course_id", "question", "answer"]
    
    if isinstance(data, dict):
        for key in keys_to_remove:
            data.pop(key, None)
        for value in data.values():
            remove_keys(value)
    elif isinstance(data, list):
        for item in data:
            remove_keys(item)

def process_json_file(input_file_path):
    with open(input_file_path, 'r') as input_file:
        json_data = json.load(input_file)
    
    remove_keys(json_data)
    
    with open(input_file_path, 'w') as output_file:
        json.dump(json_data, output_file, indent=4)

if __name__ == "__main__":
    input_json_file = "modified.json"  # Replace with the actual input JSON file path
    
    process_json_file(input_json_file)
    print("Keys removed from JSON data.")

Keys removed from JSON data.


In [20]:
#remove html tags from qna folder
import os
import re

def remove_html_tags(text):
    clean_text = re.sub(r"<.*?>", "", text)  # Remove HTML tags using regex
    return clean_text

def remove_html_tags_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:  # Specify UTF-8 encoding
        content = f.read()
    
    clean_content = remove_html_tags(content)
    
    with open(file_path, "w", encoding="utf-8") as f:  # Specify UTF-8 encoding
        f.write(clean_content)

def process_files_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            remove_html_tags_from_file(file_path)
            print(f"HTML tags removed from: {file_name}")

if __name__ == "__main__":
    txt_folder_path = "qna"  # Replace with the path to your folder of TXT files
    process_files_in_folder(txt_folder_path)

HTML tags removed from: 100.txt
HTML tags removed from: 112.txt
HTML tags removed from: 114.txt
HTML tags removed from: 123.txt
HTML tags removed from: 124.txt
HTML tags removed from: 128.txt
HTML tags removed from: 132.txt
HTML tags removed from: 133.txt
HTML tags removed from: 134.txt
HTML tags removed from: 135.txt
HTML tags removed from: 137.txt
HTML tags removed from: 138.txt
HTML tags removed from: 139.txt
HTML tags removed from: 152.txt
HTML tags removed from: 153.txt
HTML tags removed from: 154.txt
HTML tags removed from: 155.txt
HTML tags removed from: 156.txt
HTML tags removed from: 157.txt
HTML tags removed from: 158.txt
HTML tags removed from: 159.txt
HTML tags removed from: 160.txt
HTML tags removed from: 161.txt
HTML tags removed from: 162.txt
HTML tags removed from: 163.txt
HTML tags removed from: 166.txt
HTML tags removed from: 180.txt
HTML tags removed from: 181.txt
HTML tags removed from: 183.txt
HTML tags removed from: 187.txt
HTML tags removed from: 190.txt
HTML tag

In [21]:
#Remove 'alternative title: None'
import os

def remove_alternative_titles(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()

            # Remove lines containing "Alternative Title: None"
            new_lines = [line for line in lines if "Alternative Title: None" not in line]

            # Write the modified lines back to the file
            with open(file_path, "w", encoding="utf-8") as file:
                file.writelines(new_lines)

            print(f"Removed alternative titles from {filename}")

if __name__ == "__main__":
    folder_path = "qna"  # Replace with the actual folder path
    remove_alternative_titles(folder_path)

Removed alternative titles from 100.txt
Removed alternative titles from 112.txt
Removed alternative titles from 114.txt
Removed alternative titles from 123.txt
Removed alternative titles from 124.txt
Removed alternative titles from 128.txt
Removed alternative titles from 132.txt
Removed alternative titles from 133.txt
Removed alternative titles from 134.txt
Removed alternative titles from 135.txt
Removed alternative titles from 137.txt
Removed alternative titles from 138.txt
Removed alternative titles from 139.txt
Removed alternative titles from 152.txt
Removed alternative titles from 153.txt
Removed alternative titles from 154.txt
Removed alternative titles from 155.txt
Removed alternative titles from 156.txt
Removed alternative titles from 157.txt
Removed alternative titles from 158.txt
Removed alternative titles from 159.txt
Removed alternative titles from 160.txt
Removed alternative titles from 161.txt
Removed alternative titles from 162.txt
Removed alternative titles from 163.txt


In [22]:
import os

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            alternative_titles = []
            prices = []
            hours = []
            new_lines = []
            
            for line in lines:
                if line.startswith("Alternative Title: "):
                    alternative_titles.append(line)
                elif line.startswith("Price: "):
                    prices.append(line)
                elif line.startswith("Hours: "):
                    hours.append(line)
                else:
                    new_lines.append(line)
            
            if alternative_titles or prices or hours:
                new_lines = alternative_titles[:1] + prices[:1] + hours[:1] + new_lines
                
                with open(file_path, 'w', encoding='utf-8') as file:
                    file.writelines(new_lines)

if __name__ == "__main__":
    folder_path = "qna"
    process_folder(folder_path)
    print("Processing complete.")

Processing complete.


In [23]:
#Add ----- seperator between q/a pairs
import os
import codecs

def process_file(file_path):
    new_lines = []
    with codecs.open(file_path, 'r', 'utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line.strip().startswith("Question:"):
                new_lines.append("-----\n")
            new_lines.append(line)
    
    with codecs.open(file_path, 'w', 'utf-8') as file:
        file.writelines(new_lines)

def process_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                process_file(file_path)
                print(f"Processed: {file_path}")

if __name__ == "__main__":
    input_folder = "qna"  # Replace with the actual folder path
    process_folder(input_folder)
    print("Processing completed.")

Processed: qna\100.txt
Processed: qna\112.txt
Processed: qna\114.txt
Processed: qna\123.txt
Processed: qna\124.txt
Processed: qna\128.txt
Processed: qna\132.txt
Processed: qna\133.txt
Processed: qna\134.txt
Processed: qna\135.txt
Processed: qna\137.txt
Processed: qna\138.txt
Processed: qna\139.txt
Processed: qna\152.txt
Processed: qna\153.txt
Processed: qna\154.txt
Processed: qna\155.txt
Processed: qna\156.txt
Processed: qna\157.txt
Processed: qna\158.txt
Processed: qna\159.txt
Processed: qna\160.txt
Processed: qna\161.txt
Processed: qna\162.txt
Processed: qna\163.txt
Processed: qna\166.txt
Processed: qna\180.txt
Processed: qna\181.txt
Processed: qna\183.txt
Processed: qna\187.txt
Processed: qna\190.txt
Processed: qna\191.txt
Processed: qna\192.txt
Processed: qna\208.txt
Processed: qna\209.txt
Processed: qna\210.txt
Processed: qna\212.txt
Processed: qna\213.txt
Processed: qna\214.txt
Processed: qna\218.txt
Processed: qna\222.txt
Processed: qna\224.txt
Processed: qna\226.txt
Processed: 

In [24]:
#remove duplicate q/a pairs
import os
import codecs

def process_file(file_path):
    new_lines = []
    question_lines = set()
    duplicate_question = False
    
    with codecs.open(file_path, 'r', 'utf-8') as file:
        lines = file.readlines()
        
        for line in lines:
            stripped_line = line.strip()
            if stripped_line.startswith("Question:"):
                if stripped_line not in question_lines:
                    question_lines.add(stripped_line)
                    duplicate_question = False
                    new_lines.append(line)
                else:
                    duplicate_question = True
            elif stripped_line == "-----":
                if not duplicate_question:
                    new_lines.append(line)
                    duplicate_question = False
            elif not duplicate_question:
                new_lines.append(line)
    
    with codecs.open(file_path, 'w', 'utf-8') as file:
        file.writelines(new_lines)

def process_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                process_file(file_path)
                print(f"Processed: {file_path}")

if __name__ == "__main__":
    input_folder = "qna"  # Replace with the actual folder path
    process_folder(input_folder)
    print("Processing completed.")

Processed: qna\100.txt
Processed: qna\112.txt
Processed: qna\114.txt
Processed: qna\123.txt
Processed: qna\124.txt
Processed: qna\128.txt
Processed: qna\132.txt
Processed: qna\133.txt
Processed: qna\134.txt
Processed: qna\135.txt
Processed: qna\137.txt
Processed: qna\138.txt
Processed: qna\139.txt
Processed: qna\152.txt
Processed: qna\153.txt
Processed: qna\154.txt
Processed: qna\155.txt
Processed: qna\156.txt
Processed: qna\157.txt
Processed: qna\158.txt
Processed: qna\159.txt
Processed: qna\160.txt
Processed: qna\161.txt
Processed: qna\162.txt
Processed: qna\163.txt
Processed: qna\166.txt
Processed: qna\180.txt
Processed: qna\181.txt
Processed: qna\183.txt
Processed: qna\187.txt
Processed: qna\190.txt
Processed: qna\191.txt
Processed: qna\192.txt
Processed: qna\208.txt
Processed: qna\209.txt
Processed: qna\210.txt
Processed: qna\212.txt
Processed: qna\213.txt
Processed: qna\214.txt
Processed: qna\218.txt
Processed: qna\222.txt
Processed: qna\224.txt
Processed: qna\226.txt
Processed: 

In [25]:
#rename all qna files
import os
import re
import codecs

def sanitize_filename(filename):
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def extract_title(file_path):
    with codecs.open(file_path, 'r', 'utf-8') as file:
        first_line = file.readline().strip()
        match = re.match(r"^Alternative Title:\s*(.*)$", first_line)
        if match:
            return match.group(1)
    return None

def rename_file(file_path, new_name):
    base_dir = os.path.dirname(file_path)
    new_name = sanitize_filename(new_name)
    new_file_name = f"{new_name}-qna.txt"
    new_file_path = os.path.join(base_dir, new_file_name)
    
    # Avoid overwriting existing files
    counter = 1
    while os.path.exists(new_file_path):
        new_file_path = os.path.join(base_dir, f"{new_name}-qna({counter}).txt")
        counter += 1
    
    os.rename(file_path, new_file_path)
    return new_file_path

def process_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                new_name = extract_title(file_path)
                if new_name:
                    new_file_path = rename_file(file_path, new_name)
                    print(f"Renamed: {file_path} -> {new_file_path}")
                else:
                    print(f"Skipped: {file_path} (No title found)")

if __name__ == "__main__":
    input_folder = "qna"  # Replace with the actual folder path
    process_folder(input_folder)
    print("Renaming completed.")

Renamed: qna\100.txt -> qna\OSHA Confined Space Awareness Training-qna.txt
Renamed: qna\112.txt -> qna\OSHA 8 Hour HAZWOPER Refresher Training – 29 CFR 1910.120 (e)-qna.txt
Renamed: qna\114.txt -> qna\RCRA Hazardous Waste Generator Training-qna.txt
Renamed: qna\123.txt -> qna\OSHA 40 Hour HAZWOPER Training – 29 CFR 1910.120 (e)-qna.txt
Renamed: qna\124.txt -> qna\OSHA Competent Person for Fall Protection Training-qna.txt
Renamed: qna\128.txt -> qna\OSHA Asbestos Awareness Training-qna.txt
Renamed: qna\132.txt -> qna\OSHA Bloodborne Pathogens Training-qna.txt
Renamed: qna\133.txt -> qna\OSHA 8 Hour HAZWOPER Supervisor Training-qna.txt
Renamed: qna\134.txt -> qna\OSHA 8 Hour HAZWOPER Supervisor Refresher Training-qna.txt
Renamed: qna\135.txt -> qna\OSHA 24 Hour HAZWOPER - RCRA TSD Operations Training-qna.txt
Renamed: qna\137.txt -> qna\OSHA 24 Hour HAZWOPER Training – 29 CFR 1910.120 (e)-qna.txt
Renamed: qna\138.txt -> qna\OSHA 8 Hour HAZWOPER Annual Refresher - RCRA TSD Operations Train

In [26]:
#extract all objects into html files
import json
import os


def save_object_data_as_html(json_file, output_folder):
    with open(json_file) as f:
        data = json.load(f)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    for index, obj in enumerate(data, start=1):
        output_file = os.path.join(output_folder, f"object_{index}.html")

        with open(output_file, 'w', encoding='utf-8') as html_file:
            html_file.write("<html>\n<body>\n")
            html_file.write("<table>\n")

            for key, value in obj.items():
                html_file.write(f"<tr><td>{key}</td><td>{value}</td></tr>\n")

            html_file.write("</table>\n")
            html_file.write("</body>\n</html>")

        print(f"Object {index} data saved to {output_file}.")

    print(f"All object data saved as HTML files in the {output_folder} folder.")


# Usage example
json_file_path = 'modified.json'
output_folder_path = 'output'
save_object_data_as_html(json_file_path, output_folder_path)

Object 1 data saved to output\object_1.html.
Object 2 data saved to output\object_2.html.
Object 3 data saved to output\object_3.html.
Object 4 data saved to output\object_4.html.
Object 5 data saved to output\object_5.html.
Object 6 data saved to output\object_6.html.
Object 7 data saved to output\object_7.html.
Object 8 data saved to output\object_8.html.
Object 9 data saved to output\object_9.html.
Object 10 data saved to output\object_10.html.
Object 11 data saved to output\object_11.html.
Object 12 data saved to output\object_12.html.
Object 13 data saved to output\object_13.html.
Object 14 data saved to output\object_14.html.
Object 15 data saved to output\object_15.html.
Object 16 data saved to output\object_16.html.
Object 17 data saved to output\object_17.html.
Object 18 data saved to output\object_18.html.
Object 19 data saved to output\object_19.html.
Object 20 data saved to output\object_20.html.
Object 21 data saved to output\object_21.html.
Object 22 data saved to output\

Object 189 data saved to output\object_189.html.
Object 190 data saved to output\object_190.html.
Object 191 data saved to output\object_191.html.
Object 192 data saved to output\object_192.html.
Object 193 data saved to output\object_193.html.
Object 194 data saved to output\object_194.html.
Object 195 data saved to output\object_195.html.
Object 196 data saved to output\object_196.html.
Object 197 data saved to output\object_197.html.
Object 198 data saved to output\object_198.html.
Object 199 data saved to output\object_199.html.
Object 200 data saved to output\object_200.html.
Object 201 data saved to output\object_201.html.
Object 202 data saved to output\object_202.html.
Object 203 data saved to output\object_203.html.
Object 204 data saved to output\object_204.html.
Object 205 data saved to output\object_205.html.
Object 206 data saved to output\object_206.html.
Object 207 data saved to output\object_207.html.
Object 208 data saved to output\object_208.html.
Object 209 data save

Object 451 data saved to output\object_451.html.
Object 452 data saved to output\object_452.html.
Object 453 data saved to output\object_453.html.
Object 454 data saved to output\object_454.html.
Object 455 data saved to output\object_455.html.
Object 456 data saved to output\object_456.html.
Object 457 data saved to output\object_457.html.
Object 458 data saved to output\object_458.html.
Object 459 data saved to output\object_459.html.
Object 460 data saved to output\object_460.html.
Object 461 data saved to output\object_461.html.
Object 462 data saved to output\object_462.html.
Object 463 data saved to output\object_463.html.
Object 464 data saved to output\object_464.html.
Object 465 data saved to output\object_465.html.
Object 466 data saved to output\object_466.html.
Object 467 data saved to output\object_467.html.
Object 468 data saved to output\object_468.html.
Object 469 data saved to output\object_469.html.
Object 470 data saved to output\object_470.html.
Object 471 data save

Object 671 data saved to output\object_671.html.
Object 672 data saved to output\object_672.html.
Object 673 data saved to output\object_673.html.
Object 674 data saved to output\object_674.html.
Object 675 data saved to output\object_675.html.
Object 676 data saved to output\object_676.html.
Object 677 data saved to output\object_677.html.
Object 678 data saved to output\object_678.html.
Object 679 data saved to output\object_679.html.
Object 680 data saved to output\object_680.html.
Object 681 data saved to output\object_681.html.
Object 682 data saved to output\object_682.html.
Object 683 data saved to output\object_683.html.
Object 684 data saved to output\object_684.html.
Object 685 data saved to output\object_685.html.
Object 686 data saved to output\object_686.html.
Object 687 data saved to output\object_687.html.
Object 688 data saved to output\object_688.html.
Object 689 data saved to output\object_689.html.
Object 690 data saved to output\object_690.html.
Object 691 data save

Object 840 data saved to output\object_840.html.
Object 841 data saved to output\object_841.html.
Object 842 data saved to output\object_842.html.
Object 843 data saved to output\object_843.html.
Object 844 data saved to output\object_844.html.
Object 845 data saved to output\object_845.html.
Object 846 data saved to output\object_846.html.
Object 847 data saved to output\object_847.html.
Object 848 data saved to output\object_848.html.
Object 849 data saved to output\object_849.html.
Object 850 data saved to output\object_850.html.
Object 851 data saved to output\object_851.html.
Object 852 data saved to output\object_852.html.
Object 853 data saved to output\object_853.html.
Object 854 data saved to output\object_854.html.
Object 855 data saved to output\object_855.html.
Object 856 data saved to output\object_856.html.
Object 857 data saved to output\object_857.html.
Object 858 data saved to output\object_858.html.
Object 859 data saved to output\object_859.html.
Object 860 data save

...

Object 17331 data saved to output\object_17331.html.
Object 17332 data saved to output\object_17332.html.
Object 17333 data saved to output\object_17333.html.
Object 17334 data saved to output\object_17334.html.
Object 17335 data saved to output\object_17335.html.
Object 17336 data saved to output\object_17336.html.
Object 17337 data saved to output\object_17337.html.
Object 17338 data saved to output\object_17338.html.
Object 17339 data saved to output\object_17339.html.
Object 17340 data saved to output\object_17340.html.
Object 17341 data saved to output\object_17341.html.
Object 17342 data saved to output\object_17342.html.
Object 17343 data saved to output\object_17343.html.
Object 17344 data saved to output\object_17344.html.
Object 17345 data saved to output\object_17345.html.
Object 17346 data saved to output\object_17346.html.
Object 17347 data saved to output\object_17347.html.
Object 17348 data saved to output\object_17348.html.
Object 17349 data saved to output\object_17349

Object 17545 data saved to output\object_17545.html.
Object 17546 data saved to output\object_17546.html.
Object 17547 data saved to output\object_17547.html.
Object 17548 data saved to output\object_17548.html.
Object 17549 data saved to output\object_17549.html.
Object 17550 data saved to output\object_17550.html.
Object 17551 data saved to output\object_17551.html.
Object 17552 data saved to output\object_17552.html.
Object 17553 data saved to output\object_17553.html.
Object 17554 data saved to output\object_17554.html.
Object 17555 data saved to output\object_17555.html.
Object 17556 data saved to output\object_17556.html.
Object 17557 data saved to output\object_17557.html.
Object 17558 data saved to output\object_17558.html.
Object 17559 data saved to output\object_17559.html.
Object 17560 data saved to output\object_17560.html.
Object 17561 data saved to output\object_17561.html.
Object 17562 data saved to output\object_17562.html.
Object 17563 data saved to output\object_17563

Object 17805 data saved to output\object_17805.html.
Object 17806 data saved to output\object_17806.html.
Object 17807 data saved to output\object_17807.html.
Object 17808 data saved to output\object_17808.html.
Object 17809 data saved to output\object_17809.html.
Object 17810 data saved to output\object_17810.html.
Object 17811 data saved to output\object_17811.html.
Object 17812 data saved to output\object_17812.html.
Object 17813 data saved to output\object_17813.html.
Object 17814 data saved to output\object_17814.html.
Object 17815 data saved to output\object_17815.html.
Object 17816 data saved to output\object_17816.html.
Object 17817 data saved to output\object_17817.html.
Object 17818 data saved to output\object_17818.html.
Object 17819 data saved to output\object_17819.html.
Object 17820 data saved to output\object_17820.html.
Object 17821 data saved to output\object_17821.html.
Object 17822 data saved to output\object_17822.html.
Object 17823 data saved to output\object_17823

Object 17996 data saved to output\object_17996.html.
Object 17997 data saved to output\object_17997.html.
Object 17998 data saved to output\object_17998.html.
Object 17999 data saved to output\object_17999.html.
Object 18000 data saved to output\object_18000.html.
Object 18001 data saved to output\object_18001.html.
Object 18002 data saved to output\object_18002.html.
Object 18003 data saved to output\object_18003.html.
Object 18004 data saved to output\object_18004.html.
Object 18005 data saved to output\object_18005.html.
Object 18006 data saved to output\object_18006.html.
Object 18007 data saved to output\object_18007.html.
Object 18008 data saved to output\object_18008.html.
Object 18009 data saved to output\object_18009.html.
Object 18010 data saved to output\object_18010.html.
Object 18011 data saved to output\object_18011.html.
Object 18012 data saved to output\object_18012.html.
Object 18013 data saved to output\object_18013.html.
Object 18014 data saved to output\object_18014

Object 18164 data saved to output\object_18164.html.
Object 18165 data saved to output\object_18165.html.
Object 18166 data saved to output\object_18166.html.
Object 18167 data saved to output\object_18167.html.
Object 18168 data saved to output\object_18168.html.
Object 18169 data saved to output\object_18169.html.
Object 18170 data saved to output\object_18170.html.
Object 18171 data saved to output\object_18171.html.
Object 18172 data saved to output\object_18172.html.
Object 18173 data saved to output\object_18173.html.
Object 18174 data saved to output\object_18174.html.
Object 18175 data saved to output\object_18175.html.
Object 18176 data saved to output\object_18176.html.
Object 18177 data saved to output\object_18177.html.
Object 18178 data saved to output\object_18178.html.
Object 18179 data saved to output\object_18179.html.
Object 18180 data saved to output\object_18180.html.
Object 18181 data saved to output\object_18181.html.
Object 18182 data saved to output\object_18182

In [27]:
#parse all the html files into txt files
import os
from bs4 import BeautifulSoup


def parse_html_file(html_file_path):
    # Read the HTML file
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract the relevant text
    text = soup.get_text(separator='\n')

    # Remove unnecessary lines and spaces
    lines = (line.strip() for line in text.splitlines() if line.strip())
    parsed_text = '\n'.join(lines)

    return parsed_text


def save_parsed_data(html_file_path, parsed_text):
    # Create the "parsedData" directory if it doesn't exist
    output_dir = 'parsedData'
    os.makedirs(output_dir, exist_ok=True)

    # Extract the file name without extension
    file_name = os.path.splitext(os.path.basename(html_file_path))[0]

    # Save the parsed data in a text file
    output_file_path = os.path.join(output_dir, file_name + '.txt')
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(parsed_text)

    print(f"Parsed data saved for {html_file_path}")


# Directory containing HTML files
html_files_dir = 'output'

# Process each HTML file in the directory
for file_name in os.listdir(html_files_dir):
    if file_name.endswith('.html'):
        html_file_path = os.path.join(html_files_dir, file_name)

        # Parse the HTML file
        parsed_text = parse_html_file(html_file_path)

        # Save the parsed data
        save_parsed_data(html_file_path, parsed_text)

Parsed data saved for output\object_1.html
Parsed data saved for output\object_10.html
Parsed data saved for output\object_100.html
Parsed data saved for output\object_1000.html
Parsed data saved for output\object_10000.html
Parsed data saved for output\object_10001.html
Parsed data saved for output\object_10002.html
Parsed data saved for output\object_10003.html
Parsed data saved for output\object_10004.html
Parsed data saved for output\object_10005.html
Parsed data saved for output\object_10006.html
Parsed data saved for output\object_10007.html
Parsed data saved for output\object_10008.html
Parsed data saved for output\object_10009.html
Parsed data saved for output\object_1001.html
Parsed data saved for output\object_10010.html
Parsed data saved for output\object_10011.html
Parsed data saved for output\object_10012.html
Parsed data saved for output\object_10013.html
Parsed data saved for output\object_10014.html
Parsed data saved for output\object_10015.html
Parsed data saved for ou

Parsed data saved for output\object_10162.html
Parsed data saved for output\object_10163.html
Parsed data saved for output\object_10164.html
Parsed data saved for output\object_10165.html
Parsed data saved for output\object_10166.html
Parsed data saved for output\object_10167.html
Parsed data saved for output\object_10168.html
Parsed data saved for output\object_10169.html
Parsed data saved for output\object_1017.html
Parsed data saved for output\object_10170.html
Parsed data saved for output\object_10171.html
Parsed data saved for output\object_10172.html
Parsed data saved for output\object_10173.html
Parsed data saved for output\object_10174.html
Parsed data saved for output\object_10175.html
Parsed data saved for output\object_10176.html
Parsed data saved for output\object_10177.html
Parsed data saved for output\object_10178.html
Parsed data saved for output\object_10179.html
Parsed data saved for output\object_1018.html
Parsed data saved for output\object_10180.html
Parsed data sav

Parsed data saved for output\object_10325.html
Parsed data saved for output\object_10326.html
Parsed data saved for output\object_10327.html
Parsed data saved for output\object_10328.html
Parsed data saved for output\object_10329.html
Parsed data saved for output\object_1033.html
Parsed data saved for output\object_10330.html
Parsed data saved for output\object_10331.html
Parsed data saved for output\object_10332.html
Parsed data saved for output\object_10333.html
Parsed data saved for output\object_10334.html
Parsed data saved for output\object_10335.html
Parsed data saved for output\object_10336.html
Parsed data saved for output\object_10337.html
Parsed data saved for output\object_10338.html
Parsed data saved for output\object_10339.html
Parsed data saved for output\object_1034.html
Parsed data saved for output\object_10340.html
Parsed data saved for output\object_10341.html
Parsed data saved for output\object_10342.html
Parsed data saved for output\object_10343.html
Parsed data sav

Parsed data saved for output\object_10491.html
Parsed data saved for output\object_10492.html
Parsed data saved for output\object_10493.html
Parsed data saved for output\object_10494.html
Parsed data saved for output\object_10495.html
Parsed data saved for output\object_10496.html
Parsed data saved for output\object_10497.html
Parsed data saved for output\object_10498.html
Parsed data saved for output\object_10499.html
Parsed data saved for output\object_105.html
Parsed data saved for output\object_1050.html
Parsed data saved for output\object_10500.html
Parsed data saved for output\object_10501.html
Parsed data saved for output\object_10502.html
Parsed data saved for output\object_10503.html
Parsed data saved for output\object_10504.html
Parsed data saved for output\object_10505.html
Parsed data saved for output\object_10506.html
Parsed data saved for output\object_10507.html
Parsed data saved for output\object_10508.html
Parsed data saved for output\object_10509.html
Parsed data save

Parsed data saved for output\object_10649.html
Parsed data saved for output\object_1065.html
Parsed data saved for output\object_10650.html
Parsed data saved for output\object_10651.html
Parsed data saved for output\object_10652.html
Parsed data saved for output\object_10653.html
Parsed data saved for output\object_10654.html
Parsed data saved for output\object_10655.html
Parsed data saved for output\object_10656.html
Parsed data saved for output\object_10657.html
Parsed data saved for output\object_10658.html
Parsed data saved for output\object_10659.html
Parsed data saved for output\object_1066.html
Parsed data saved for output\object_10660.html
Parsed data saved for output\object_10661.html
Parsed data saved for output\object_10662.html
Parsed data saved for output\object_10663.html
Parsed data saved for output\object_10664.html
Parsed data saved for output\object_10665.html
Parsed data saved for output\object_10666.html
Parsed data saved for output\object_10667.html
Parsed data sav

...

Parsed data saved for output\object_9316.html
Parsed data saved for output\object_9317.html
Parsed data saved for output\object_9318.html
Parsed data saved for output\object_9319.html
Parsed data saved for output\object_932.html
Parsed data saved for output\object_9320.html
Parsed data saved for output\object_9321.html
Parsed data saved for output\object_9322.html
Parsed data saved for output\object_9323.html
Parsed data saved for output\object_9324.html
Parsed data saved for output\object_9325.html
Parsed data saved for output\object_9326.html
Parsed data saved for output\object_9327.html
Parsed data saved for output\object_9328.html
Parsed data saved for output\object_9329.html
Parsed data saved for output\object_933.html
Parsed data saved for output\object_9330.html
Parsed data saved for output\object_9331.html
Parsed data saved for output\object_9332.html
Parsed data saved for output\object_9333.html
Parsed data saved for output\object_9334.html
Parsed data saved for output\object_

Parsed data saved for output\object_9488.html
Parsed data saved for output\object_9489.html
Parsed data saved for output\object_949.html
Parsed data saved for output\object_9490.html
Parsed data saved for output\object_9491.html
Parsed data saved for output\object_9492.html
Parsed data saved for output\object_9493.html
Parsed data saved for output\object_9494.html
Parsed data saved for output\object_9495.html
Parsed data saved for output\object_9496.html
Parsed data saved for output\object_9497.html
Parsed data saved for output\object_9498.html
Parsed data saved for output\object_9499.html
Parsed data saved for output\object_95.html
Parsed data saved for output\object_950.html
Parsed data saved for output\object_9500.html
Parsed data saved for output\object_9501.html
Parsed data saved for output\object_9502.html
Parsed data saved for output\object_9503.html
Parsed data saved for output\object_9504.html
Parsed data saved for output\object_9505.html
Parsed data saved for output\object_95

Parsed data saved for output\object_965.html
Parsed data saved for output\object_9650.html
Parsed data saved for output\object_9651.html
Parsed data saved for output\object_9652.html
Parsed data saved for output\object_9653.html
Parsed data saved for output\object_9654.html
Parsed data saved for output\object_9655.html
Parsed data saved for output\object_9656.html
Parsed data saved for output\object_9657.html
Parsed data saved for output\object_9658.html
Parsed data saved for output\object_9659.html
Parsed data saved for output\object_966.html
Parsed data saved for output\object_9660.html
Parsed data saved for output\object_9661.html
Parsed data saved for output\object_9662.html
Parsed data saved for output\object_9663.html
Parsed data saved for output\object_9664.html
Parsed data saved for output\object_9665.html
Parsed data saved for output\object_9666.html
Parsed data saved for output\object_9667.html
Parsed data saved for output\object_9668.html
Parsed data saved for output\object_

Parsed data saved for output\object_9814.html
Parsed data saved for output\object_9815.html
Parsed data saved for output\object_9816.html
Parsed data saved for output\object_9817.html
Parsed data saved for output\object_9818.html
Parsed data saved for output\object_9819.html
Parsed data saved for output\object_982.html
Parsed data saved for output\object_9820.html
Parsed data saved for output\object_9821.html
Parsed data saved for output\object_9822.html
Parsed data saved for output\object_9823.html
Parsed data saved for output\object_9824.html
Parsed data saved for output\object_9825.html
Parsed data saved for output\object_9826.html
Parsed data saved for output\object_9827.html
Parsed data saved for output\object_9828.html
Parsed data saved for output\object_9829.html
Parsed data saved for output\object_983.html
Parsed data saved for output\object_9830.html
Parsed data saved for output\object_9831.html
Parsed data saved for output\object_9832.html
Parsed data saved for output\object_

Parsed data saved for output\object_9985.html
Parsed data saved for output\object_9986.html
Parsed data saved for output\object_9987.html
Parsed data saved for output\object_9988.html
Parsed data saved for output\object_9989.html
Parsed data saved for output\object_999.html
Parsed data saved for output\object_9990.html
Parsed data saved for output\object_9991.html
Parsed data saved for output\object_9992.html
Parsed data saved for output\object_9993.html
Parsed data saved for output\object_9994.html
Parsed data saved for output\object_9995.html
Parsed data saved for output\object_9996.html
Parsed data saved for output\object_9997.html
Parsed data saved for output\object_9998.html
Parsed data saved for output\object_9999.html


In [28]:
#duplicates remove
import os
import filecmp

def remove_duplicate_files(directory):
    files = os.listdir(directory)
    duplicate_files = set()

    # Compare file contents
    for i in range(len(files)):
        file1 = os.path.join(directory, files[i])
        if file1 in duplicate_files:
            continue

        for j in range(i+1, len(files)):
            file2 = os.path.join(directory, files[j])
            if file2 in duplicate_files:
                continue

            if filecmp.cmp(file1, file2):
                duplicate_files.add(file2)
                os.remove(file2)
    
    print(f"{len(duplicate_files)} duplicate files removed.")

# Example usage
directory_path = 'parsedData'
remove_duplicate_files(directory_path)

17500 duplicate files removed.


In [29]:
#rename all files according to title-page Type/alt_title
import os

def rename_files(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
                lines = file.readlines()
                if len(lines) >= 2:
                    title = lines[1].strip()
                    page_type = lines[-1].strip()

            # Construct the new filename
            new_filename = f"{title}-{page_type}.txt"
            new_filename = new_filename.replace("/", "-")  # Remove any forward slashes
            new_filename = new_filename.replace(":", " -")  # Remove colons
            new_filename = new_filename.replace("?", "")  # Remove question marks

            # Ensure the new filename is unique in the directory
            count = 1
            while True:
                new_filename_with_count = new_filename if count == 1 else f"{new_filename[:-4]}_{count}.txt"
                new_file_path = os.path.join(directory, new_filename_with_count)

                if not os.path.exists(new_file_path):
                    break
                count += 1

            os.rename(file_path, new_file_path)
            print(f"File '{filename}' renamed to '{os.path.basename(new_file_path)}'.")

# Specify the directory containing the text files
directory_path = 'parsedData'

# Call the function to rename the files
rename_files(directory_path)

File 'object_1.txt' renamed to 'About HAZWOPER OSHA Training-Static.txt'.
File 'object_10.txt' renamed to 'Silica Exposure Awareness for Occupational Safety-Blogs.txt'.
File 'object_100.txt' renamed to 'Protecting Workers from Heat Stress-Blogs.txt'.
File 'object_1000.txt' renamed to 'Learning Objectives-RCRA Hazardous Waste Generator Training.txt'.
File 'object_10000.txt' renamed to 'Group Training-DOT Hazmat Carrier Requirements - Highway.txt'.
File 'object_10001.txt' renamed to 'Group Training-DOT Hazmat Carrier Requirements - Air-IATA.txt'.
File 'object_1001.txt' renamed to 'Syllabus-RCRA Hazardous Waste Generator Training.txt'.
File 'object_10010.txt' renamed to 'Group Training-DOT Hazmat Carrier Requirements - Rail.txt'.
File 'object_10019.txt' renamed to 'Group Training-DOT Hazmat Carrier Requirements - Vessel.txt'.
File 'object_1002.txt' renamed to 'Overview-OSHA Hydrogen Sulfide Awareness Training.txt'.
File 'object_10028.txt' renamed to 'Group Training-OSHA 40 Hour HAZWOPER T

File 'object_1140.txt' renamed to 'Overview-OSHA Scaffolding Safety Training.txt'.
File 'object_1141.txt' renamed to 'Learning Objectives-OSHA Scaffolding Safety Training.txt'.
File 'object_11419.txt' renamed to 'Learning Objectives-OSHA Lockout Tagout (LOTO) Training.txt'.
File 'object_1142.txt' renamed to 'Syllabus-OSHA Scaffolding Safety Training.txt'.
File 'object_11420.txt' renamed to 'Syllabus-OSHA Lockout Tagout (LOTO) Training.txt'.
File 'object_11421.txt' renamed to 'Accreditation & Requirements-OSHA Lockout Tagout (LOTO) Training.txt'.
File 'object_11422.txt' renamed to 'Group Training-OSHA Lockout Tagout (LOTO) Training.txt'.
File 'object_11423.txt' renamed to 'Resources-Course Page_9.txt'.
File 'object_1143.txt' renamed to 'Accreditation & Requirements-OSHA Scaffolding Safety Training.txt'.
File 'object_1144.txt' renamed to 'Resources-Course Page_10.txt'.
File 'object_11520.txt' renamed to 'The Role of Lockout Tagout in Electrical Safety-Blogs.txt'.
File 'object_11521.txt' 

File 'object_12157.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training - The Hazmat Table.txt'.
File 'object_12166.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training  - Shipping Papers.txt'.
File 'object_12175.txt' renamed to 'SCORM Package-OSHA Silica Awareness Training.txt'.
File 'object_12181.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training - Packaging.txt'.
File 'object_12190.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training - Markings.txt'.
File 'object_12199.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training - Packaging_2.txt'.
File 'object_122.txt' renamed to 'Core Elements of an Effective Safety Program-Blogs.txt'.
File 'object_12208.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training - Placarding.txt'.
File 'object_12217.txt' renamed to 'SCORM Package-DOT Hazmat Function Specific Training - Security Awareness.txt'.
File 'object_12226.txt' renamed to 'SCORM Package-DOT H

File 'object_1260.txt' renamed to 'Chad Baker, Petrochemical Engineer-Testimonials.txt'.
File 'object_1261.txt' renamed to 'Diana Bayram, Management Accountant-Testimonials.txt'.
File 'object_1262.txt' renamed to 'M. K. Gerald, Customer Services Executive-Testimonials.txt'.
File 'object_1263.txt' renamed to 'Sara Evans, HR Assistant Manager-Testimonials.txt'.
File 'object_1264.txt' renamed to 'Mike Andreas, Small Business Owner-Testimonials.txt'.
File 'object_1265.txt' renamed to 'Giles Davis, Operations Manager-Testimonials.txt'.
File 'object_1266.txt' renamed to 'Susan Goodman, Factory Floor Manager-Testimonials.txt'.
File 'object_1267.txt' renamed to 'Cal Stevens, Site Supervisor-Testimonials.txt'.
File 'object_1268.txt' renamed to 'Toni de Mel, Manager - Employee Health & Safety-Testimonials.txt'.
File 'object_1269.txt' renamed to 'David McKenna, EHS Senior Manager-Testimonials.txt'.
File 'object_1270.txt' renamed to 'Lead Instructor-Static.txt'.
File 'object_1271.txt' renamed to '

File 'object_15323.txt' renamed to 'Overview-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt'.
File 'object_15324.txt' renamed to 'Learning Objectives-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt'.
File 'object_15325.txt' renamed to 'Syllabus-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt'.
File 'object_15326.txt' renamed to 'Accreditation & Requirements-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt'.
File 'object_15327.txt' renamed to 'Group Training-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt'.
File 'object_15328.txt' renamed to 'SCORM Package-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt'.
File 'object_15329.txt' renamed to 'Resources-Course Page_14.txt'.
File 'object_15400.txt' renamed to 'Overview-

File 'object_16784.txt' renamed to 'Acreditación y Requisitos-online-courses-persona-competente-de-osha-para-capacitacin-en-excavacin-apertura-de-zanjas-y-apuntalamiento-acreditación-y-requisitos.txt'.
File 'object_16785.txt' renamed to 'Paquete SCORM-online-courses-persona-competente-de-osha-para-capacitacin-en-excavacin-apertura-de-zanjas-y-apuntalamiento-paquete-scorm.txt'.
File 'object_16786.txt' renamed to 'Recursos-Course Page.txt'.
File 'object_16823.txt' renamed to 'Overview-OSHA Fire Safety Training.txt'.
File 'object_16824.txt' renamed to 'Learning Objectives-OSHA Fire Safety Training.txt'.
File 'object_16825.txt' renamed to 'Syllabus-OSHA Fire Safety Training.txt'.
File 'object_16826.txt' renamed to 'Accreditation & Requirements-OSHA Fire Safety Training.txt'.
File 'object_16827.txt' renamed to 'Group Training-OSHA Fire Safety Training.txt'.
File 'object_16828.txt' renamed to 'SCORM Package-OSHA Fire Safety Training.txt'.
File 'object_16829.txt' renamed to 'Resources-Course 

File 'object_2630.txt' renamed to 'Overview-OSHA 40 Hour HAZWOPER Training – 29 CFR 1910.120 (e).txt'.
File 'object_2631.txt' renamed to 'Learning Objectives-OSHA 40 Hour HAZWOPER Training – 29 CFR 1910.120 (e).txt'.
File 'object_2632.txt' renamed to 'Syllabus-OSHA 40 Hour HAZWOPER Training – 29 CFR 1910.120 (e).txt'.
File 'object_2633.txt' renamed to 'Accreditation & Requirements-OSHA 40 Hour HAZWOPER Training – 29 CFR 1910.120 (e).txt'.
File 'object_2634.txt' renamed to 'Resources-Course Page_22.txt'.
File 'object_2810.txt' renamed to 'Accreditation & Requirements-RCRA Hazardous Waste Generator Training.txt'.
File 'object_2811.txt' renamed to 'Resources-Course Page_23.txt'.
File 'object_3050.txt' renamed to 'Overview-OSHA 8 Hour HAZWOPER Supervisor Refresher Training.txt'.
File 'object_3051.txt' renamed to 'Learning Objectives-OSHA 8 Hour HAZWOPER Supervisor Refresher Training.txt'.
File 'object_3052.txt' renamed to 'Syllabus-OSHA 8 Hour HAZWOPER Supervisor Refresher Training.txt'.
F

File 'object_6187.txt' renamed to 'Resources-Course Page_39.txt'.
File 'object_6228.txt' renamed to 'Overview-DOT Hazmat Function Specific Training - Packaging_2.txt'.
File 'object_6229.txt' renamed to 'Learning Objectives-DOT Hazmat Function Specific Training - Packaging_2.txt'.
File 'object_6230.txt' renamed to 'Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt'.
File 'object_6231.txt' renamed to 'Accreditation & Requirements-DOT Hazmat Function Specific Training - Packaging_2.txt'.
File 'object_6232.txt' renamed to 'Resources-Course Page_40.txt'.
File 'object_6273.txt' renamed to 'Overview-DOT Hazmat Function Specific Training - Placarding.txt'.
File 'object_6274.txt' renamed to 'Learning Objectives-DOT Hazmat Function Specific Training - Placarding.txt'.
File 'object_6275.txt' renamed to 'Syllabus-DOT Hazmat Function Specific Training - Placarding.txt'.
File 'object_6276.txt' renamed to 'Accreditation & Requirements-DOT Hazmat Function Specific Training - Placarding.

In [30]:
#remove resources files
import os

def remove_files_with_pattern(directory):
    files_to_remove = []
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            # Check if the filename matches the English version
            if filename.startswith("Resources-Course Page") and filename.endswith(".txt"):
                files_to_remove.append(file_path)
            # Check if the filename matches the Spanish version
            elif filename == "Recursos-Course Page.txt":
                files_to_remove.append(file_path)

    for file_path in files_to_remove:
        os.remove(file_path)
        print(f"File '{os.path.basename(file_path)}' removed.")

# Specify the directory containing the text files
directory_path = 'parsedData'

# Call the function to remove files with the specified pattern
remove_files_with_pattern(directory_path)

File 'Recursos-Course Page.txt' removed.
File 'Resources-Course Page.txt' removed.
File 'Resources-Course Page_10.txt' removed.
File 'Resources-Course Page_11.txt' removed.
File 'Resources-Course Page_12.txt' removed.
File 'Resources-Course Page_13.txt' removed.
File 'Resources-Course Page_14.txt' removed.
File 'Resources-Course Page_15.txt' removed.
File 'Resources-Course Page_16.txt' removed.
File 'Resources-Course Page_17.txt' removed.
File 'Resources-Course Page_18.txt' removed.
File 'Resources-Course Page_19.txt' removed.
File 'Resources-Course Page_2.txt' removed.
File 'Resources-Course Page_20.txt' removed.
File 'Resources-Course Page_21.txt' removed.
File 'Resources-Course Page_22.txt' removed.
File 'Resources-Course Page_23.txt' removed.
File 'Resources-Course Page_24.txt' removed.
File 'Resources-Course Page_25.txt' removed.
File 'Resources-Course Page_26.txt' removed.
File 'Resources-Course Page_27.txt' removed.
File 'Resources-Course Page_28.txt' removed.
File 'Resources-Co

In [31]:
#remove duplicate passage in all overview files
import os

def remove_content_between_lines(file_path, start_line, end_line):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(file_path, 'w', encoding='utf-8') as file:
        copy = True
        for line in lines:
            if start_line in line:
                copy = False
            if copy:
                file.write(line)
            if end_line in line:
                copy = True
                file.write(line)  # Write the line containing "meta_description" back

def process_files(folder_path):
    start_line = "24/7 Support"
    end_line = "meta_description"

    for filename in os.listdir(folder_path):
        if filename.startswith("Overview-"):
            file_path = os.path.join(folder_path, filename)
            remove_content_between_lines(file_path, start_line, end_line)
            print(f"Content removed from '{filename}'.")

# Specify the folder containing the text files
folder_path = 'parsedData'

# Call the function to process the files
process_files(folder_path)

Content removed from 'Overview-Arc Flash Safety Training (NFPA 70E).txt'.
Content removed from 'Overview-DOT Hazmat Carrier Requirements - Air-IATA.txt'.
Content removed from 'Overview-DOT Hazmat Carrier Requirements - Highway.txt'.
Content removed from 'Overview-DOT Hazmat Carrier Requirements - Rail.txt'.
Content removed from 'Overview-DOT Hazmat Carrier Requirements - Vessel.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training  - Shipping Papers.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training - Markings.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training - Packaging.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training - Packaging_2.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training - Placarding.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training - Security Awareness.txt'.
Content removed from 'Overview-DOT Hazmat Function Specific Training - Th

In [32]:
#bring alt_title up after title for specific keys
import os

def move_last_2_lines(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    if len(lines) >= 2:
        last_2_lines = lines[-2:]
        lines = lines[:-2]
        lines.insert(2, last_2_lines[0].strip() + '\n')
        lines.insert(3, last_2_lines[1].strip() + '\n')

        with open(file_path, 'w', encoding='utf-8') as file:
            file.writelines(lines)

def process_files_in_folder(folder_path):
    prefixes = ["Syllabus-", "Accreditation & Requirements", "Learning Objectives", "Overview-",'Group Training','SCORM Package-']
    for filename in os.listdir(folder_path):
        for prefix in prefixes:
            if filename.startswith(prefix) and filename.endswith(".txt"):
                file_path = os.path.join(folder_path, filename)
                move_last_2_lines(file_path)

if __name__ == "__main__":
    folder_path = "parsedData"
    process_files_in_folder(folder_path)

In [33]:
#copy syllabus files to a new folder
import os
import shutil

def copy_syllabus_files(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.startswith("Syllabus-") and filename.endswith('.txt'):
            source_file = os.path.join(input_folder, filename)
            target_file = os.path.join(output_folder, filename)
            shutil.copy(source_file, target_file)
            print(f"Copied file: {filename}")

if __name__ == "__main__":
    input_folder = "parsedData"  # Replace this with the actual path to your folder of txt files
    output_folder = "syllabus"  # Replace this with the actual path for the 'syllabus' folder
    copy_syllabus_files(input_folder, output_folder)

Copied file: Syllabus-Arc Flash Safety Training (NFPA 70E).txt
Copied file: Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt
Copied file: Syllabus-DOT Hazmat Carrier Requirements - Highway.txt
Copied file: Syllabus-DOT Hazmat Carrier Requirements - Rail.txt
Copied file: Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training - Markings.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training - Packaging.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training - Placarding.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training - Security Awareness.txt
Copied file: Syllabus-DOT Hazmat Function Specific Training - The Hazmat Table.txt
Copied file: Syllabus-DOT Hazmat Transportation - General Awareness Training.txt
Copied file: Syllabus-DOT Hazm

In [34]:
#remove quizzes section
import os

def remove_quiz_description(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(file_path, 'w', encoding='utf-8') as file:
        in_quiz = False
        for line in lines:
            if in_quiz and "meta_description" in line:
                in_quiz = False
                file.write(line)
            elif "Quizzes" in line:
                in_quiz = True
            elif not in_quiz:
                file.write(line)

# Specify the folder containing the text files
folder_path = 'syllabus'

# Process each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        remove_quiz_description(file_path)
        print(f"Quiz description removed from '{filename}'.")


Quiz description removed from 'Syllabus-Arc Flash Safety Training (NFPA 70E).txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Carrier Requirements - Highway.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Carrier Requirements - Rail.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Function Specific Training - Markings.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Function Specific Training - Packaging.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Function Specific Training - Placarding.txt'.
Quiz description removed from 'Syllabus-DOT Hazmat Function Specific Training -

In [35]:
#add final examination line to syllabus
import os

def add_final_examination(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    if "Final Examination\n" in lines:
        return  # "Final Examination" already exists, do nothing

    with open(file_path, 'w', encoding='utf-8') as file:
        for line in lines:
            if "meta_description" in line:
                file.write("\nFinal Examination\n")
            file.write(line)

# Specify the folder containing the text files
folder_path = 'syllabus'

# Process each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        add_final_examination(file_path)
        print(f"'Final Examination' line added to '{filename}'.")

'Final Examination' line added to 'Syllabus-Arc Flash Safety Training (NFPA 70E).txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Carrier Requirements - Highway.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Carrier Requirements - Rail.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Function Specific Training - Markings.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Function Specific Training - Packaging.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt'.
'Final Examination' line added to 'Syllabus-DOT Hazmat Function Specific Training - Placarding.txt'.
'Final Examination' line added to 'Syll

In [36]:
#seperate module and lesson format syllabus files
import os
import shutil

def count_word_occurrences(file_path, word):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    return content.lower().count(word.lower())

def move_files_with_word_occurrences(src_folder, dest_folder, word, min_occurrences=2):
    os.makedirs(dest_folder, exist_ok=True)

    for filename in os.listdir(src_folder):
        file_path = os.path.join(src_folder, filename)
        if os.path.isfile(file_path):
            occurrences = count_word_occurrences(file_path, word)
            if occurrences >= min_occurrences:
                new_file_path = os.path.join(dest_folder, filename)
                shutil.move(file_path, new_file_path)
                print(f"File '{filename}' moved to '{dest_folder}'.")

# Specify the folder containing the text files
source_folder_path = 'syllabus'

# Specify the folder to store files with 2 or more occurrences of the word "module"
destination_folder_path = 'syllabus 2'

# Specify the word to search for
word_to_search = "module"

# Call the function to move files with 2 or more occurrences of the word "module"
move_files_with_word_occurrences(source_folder_path, destination_folder_path, word_to_search, min_occurrences=2)

File 'Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Carrier Requirements - Highway.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Carrier Requirements - Rail.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training - Markings.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training - Packaging.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training - Placarding.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training - Security Awareness.txt' moved to 'syllabus 2'.
File 'Syllabus-DOT Hazmat Function Specific Training - The Hazmat Table.txt' moved to '

In [37]:
#remove lesson descriptons
import os

def remove_lesson_description(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(file_path, 'w', encoding='utf-8') as file:
        in_lesson = False
        for line in lines:
            if line.startswith("Lesson "):
                in_lesson = True
                file.write(line)
            elif "Final Examination" in line:
                in_lesson = False
                file.write(line)
            elif not in_lesson:
                file.write(line)

# Specify the folder containing the text files
folder_path = 'syllabus'

# Process each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        remove_lesson_description(file_path)
        print(f"Lesson description removed from '{filename}'.")


Lesson description removed from 'Syllabus-Arc Flash Safety Training (NFPA 70E).txt'.
Lesson description removed from 'Syllabus-Lithium Batteries Awareness Training.txt'.
Lesson description removed from 'Syllabus-Mold Prevention, Remediation, and Removal for Construction Training.txt'.
Lesson description removed from 'Syllabus-OSHA Asbestos Awareness Training.txt'.
Lesson description removed from 'Syllabus-OSHA Bloodborne Pathogens Training.txt'.
Lesson description removed from 'Syllabus-OSHA Competent Person for Excavation, Trenching, and Shoring Training.txt'.
Lesson description removed from 'Syllabus-OSHA Confined Space Awareness Training.txt'.
Lesson description removed from 'Syllabus-OSHA Fire Safety Training.txt'.
Lesson description removed from 'Syllabus-OSHA Hazard Communication with GHS Training.txt'.
Lesson description removed from 'Syllabus-OSHA Hydrogen Sulfide Awareness Training.txt'.
Lesson description removed from 'Syllabus-OSHA Lead Awareness Training.txt'.
Lesson descri

In [38]:
#remove module descriptions
import os
import re

def remove_lines_between_modules(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    in_module = False
    for line in lines:
        if re.match(r'^Module \d+:', line.strip()):
            in_module = True
            new_lines.append(line)
        elif in_module and "Final Examination" in line:
            in_module = False
            new_lines.append(line)
        elif not in_module:
            new_lines.append(line)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

def process_files(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            remove_lines_between_modules(file_path)
            print(f"Content removed from '{filename}'.")

# Specify the folder containing the text files
folder_path = 'syllabus 2'

# Call the function to process the files
process_files(folder_path)

Content removed from 'Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt'.
Content removed from 'Syllabus-DOT Hazmat Carrier Requirements - Highway.txt'.
Content removed from 'Syllabus-DOT Hazmat Carrier Requirements - Rail.txt'.
Content removed from 'Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training - Markings.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training - Packaging.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training - Placarding.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training - Security Awareness.txt'.
Content removed from 'Syllabus-DOT Hazmat Function Specific Training - The Hazmat Table.txt'.
Content removed from 'Syllabus-DOT Hazmat Transportat

In [39]:
#merge all syllabus files together
import os
import shutil

def copy_files(source_folder, destination_folder):
    os.makedirs(destination_folder, exist_ok=True)
    
    for filename in os.listdir(source_folder):
        source_file_path = os.path.join(source_folder, filename)
        destination_file_path = os.path.join(destination_folder, filename)
        
        if os.path.isfile(source_file_path):
            shutil.copy(source_file_path, destination_file_path)
            print(f"File '{filename}' copied to '{destination_folder}'.")

# Specify the source folder containing the files
source_folder = 'syllabus 2'

# Specify the destination folder where the files will be copied
destination_folder = 'syllabus'

# Call the function to copy files from the source folder to the destination folder
copy_files(source_folder, destination_folder)

File 'Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Carrier Requirements - Highway.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Carrier Requirements - Rail.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training - Markings.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training - Packaging.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training - Placarding.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training - Security Awareness.txt' copied to 'syllabus'.
File 'Syllabus-DOT Hazmat Function Specific Training - The Hazmat Table.txt' copied to 'syllabus'

In [40]:
#extract links and add it in the last
import os

def copy_links_from_file(file_path):
    links = []
    with open(file_path, 'r') as file:
        for line in file:
            if "online-courses/" in line:
                link_start = line.find("online-courses/")
                link = line[link_start:].strip()
                links.append(link)
    return links

def modify_file(file_path, links):
    modified_lines = []
    with open(file_path, 'r') as file:
        found_module = False
        for line in file:
            modified_lines.append(line)
            if not found_module:
                if "Module 3:" in line or "Lesson 3:" in line:
                    found_module = True
    
    if found_module:
        with open(file_path, 'w') as file:
            for line in modified_lines:
                if "Module 3:" in line or "Lesson 3:" in line:
                    break
                file.write(line)
            file.write(f"To see the full course or outline go to the webpage https://hazwoper-osha.com/{links[0]}\n")

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            links = copy_links_from_file(file_path)
            if links:
                modify_file(file_path, links)

if __name__ == "__main__":
    folder_path = "syllabus"
    process_folder(folder_path)

In [41]:
#extract links and add it in the last before meta_description
import os

def extract_links_from_file(file_path):
    links = []
    with open(file_path, 'r') as file:
        for line in file:
            if "online-courses/" in line:
                link_start = line.find("online-courses/")
                link = line[link_start:].strip()
                links.append(link)
    return links

def modify_file_with_links(file_path, links):
    temp_file_path = file_path + ".temp"
    with open(file_path, 'r') as file, open(temp_file_path, 'w') as temp_file:
        found_meta_description = False
        for line in file:
            if "meta_description" in line:
                found_meta_description = True
                temp_file.write(f"To see the full course or outline go to the webpage https://hazwoper-osha.com/{links[0]}\n")
            temp_file.write(line)
    
    os.remove(file_path)
    os.rename(temp_file_path, file_path)

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            links = extract_links_from_file(file_path)
            if links:
                modify_file_with_links(file_path, links)

if __name__ == "__main__":
    folder_path = "syllabus"  # Replace with the actual path to your folder
    process_folder(folder_path)

In [42]:
#remove the introduction line
import os

def remove_introduction_line(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    with open(file_path, 'w', encoding='utf-8') as file:
        for line in lines:
            if not line.strip().endswith(": Introduction"):
                file.write(line)

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            remove_introduction_line(file_path)
            print(f"Processed: {filename}")

if __name__ == "__main__":
    folder_path = "syllabus"  # Replace this with the actual folder path
    process_folder(folder_path)
    print("Introduction lines removed from txt files.")

Processed: Syllabus-Arc Flash Safety Training (NFPA 70E).txt
Processed: Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt
Processed: Syllabus-DOT Hazmat Carrier Requirements - Highway.txt
Processed: Syllabus-DOT Hazmat Carrier Requirements - Rail.txt
Processed: Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt
Processed: Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt
Processed: Syllabus-DOT Hazmat Function Specific Training - Markings.txt
Processed: Syllabus-DOT Hazmat Function Specific Training - Packaging.txt
Processed: Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt
Processed: Syllabus-DOT Hazmat Function Specific Training - Placarding.txt
Processed: Syllabus-DOT Hazmat Function Specific Training - Security Awareness.txt
Processed: Syllabus-DOT Hazmat Function Specific Training - The Hazmat Table.txt
Processed: Syllabus-DOT Hazmat Transportation - General Awareness Training.txt
Processed: Syllabus-DOT Hazmat Transportation – Advanced

In [43]:
#copy syllabus to parsedData
import os
import shutil

def copy_files(src_folder, dest_folder):
    try:
        os.makedirs(dest_folder, exist_ok=True)

        for filename in os.listdir(src_folder):
            src_file_path = os.path.join(src_folder, filename)
            dest_file_path = os.path.join(dest_folder, filename)

            if os.path.isfile(src_file_path) and os.path.splitext(filename)[1] == ".txt":
                shutil.copyfile(src_file_path, dest_file_path)
                print(f"Copied: {filename}")

        print("All text files have been copied successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    source_folder = "syllabus"
    destination_folder = "parsedData"

    copy_files(source_folder, destination_folder)

Copied: Syllabus-Arc Flash Safety Training (NFPA 70E).txt
Copied: Syllabus-DOT Hazmat Carrier Requirements - Air-IATA.txt
Copied: Syllabus-DOT Hazmat Carrier Requirements - Highway.txt
Copied: Syllabus-DOT Hazmat Carrier Requirements - Rail.txt
Copied: Syllabus-DOT Hazmat Carrier Requirements - Vessel.txt
Copied: Syllabus-DOT Hazmat Function Specific Training  - Shipping Papers.txt
Copied: Syllabus-DOT Hazmat Function Specific Training - Markings.txt
Copied: Syllabus-DOT Hazmat Function Specific Training - Packaging.txt
Copied: Syllabus-DOT Hazmat Function Specific Training - Packaging_2.txt
Copied: Syllabus-DOT Hazmat Function Specific Training - Placarding.txt
Copied: Syllabus-DOT Hazmat Function Specific Training - Security Awareness.txt
Copied: Syllabus-DOT Hazmat Function Specific Training - The Hazmat Table.txt
Copied: Syllabus-DOT Hazmat Transportation - General Awareness Training.txt
Copied: Syllabus-DOT Hazmat Transportation – Advanced General Awareness Training.txt
Copied: Sy

In [44]:
#copy files from qna folder to parsedData
import os
import shutil

def copy_txt_files(source_folder, destination_folder):
    for root, _, files in os.walk(source_folder):
        for file in files:
            if file.endswith(".txt"):
                source_path = os.path.join(root, file)
                destination_path = os.path.join(destination_folder, file)
                shutil.copy2(source_path, destination_path)
                print(f"Copied: {source_path} -> {destination_path}")

if __name__ == "__main__":
    source_folder = "qna"  # Replace with the actual source folder path
    destination_folder = "parsedData"  # Replace with the actual destination folder path
    
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    copy_txt_files(source_folder, destination_folder)
    print("Copying completed.")

Copied: qna\191.txt -> parsedData\191.txt
Copied: qna\224.txt -> parsedData\224.txt
Copied: qna\Arc Flash Safety Training (NFPA 70E)-qna.txt -> parsedData\Arc Flash Safety Training (NFPA 70E)-qna.txt
Copied: qna\DOT Hazmat Carrier Requirements_ Air_IATA-qna.txt -> parsedData\DOT Hazmat Carrier Requirements_ Air_IATA-qna.txt
Copied: qna\DOT Hazmat Carrier Requirements_ Highway-qna.txt -> parsedData\DOT Hazmat Carrier Requirements_ Highway-qna.txt
Copied: qna\DOT Hazmat Carrier Requirements_ Rail-qna.txt -> parsedData\DOT Hazmat Carrier Requirements_ Rail-qna.txt
Copied: qna\DOT Hazmat Carrier Requirements_ Vessel-qna.txt -> parsedData\DOT Hazmat Carrier Requirements_ Vessel-qna.txt
Copied: qna\DOT Hazmat Function Specific Training _ Shipping Papers-qna.txt -> parsedData\DOT Hazmat Function Specific Training _ Shipping Papers-qna.txt
Copied: qna\DOT Hazmat Function Specific Training_ Markings-qna.txt -> parsedData\DOT Hazmat Function Specific Training_ Markings-qna.txt
Copied: qna\DOT Ha

In [45]:
# add the seperator line in the end of all files
import os

def add_line_to_end(file_path):
    with open(file_path, 'a') as file:
        file.write("\n####################\n")

def process_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            add_line_to_end(file_path)

if __name__ == "__main__":
    folder_path = "parsedData"
    process_files_in_folder(folder_path)

In [46]:
#add extra files to parsedData folder
import os
import shutil

def copy_files(source_folder1, source_folder2, destination_folder):
    # Get all files from the first source folder
    for filename in os.listdir(source_folder1):
        if filename.endswith(".txt"):
            source_file = os.path.join(source_folder1, filename)
            destination_file = os.path.join(destination_folder, filename)
            shutil.copyfile(source_file, destination_file)

    # Get all files from the second source folder
    for filename in os.listdir(source_folder2):
        if filename.endswith(".txt"):
            source_file = os.path.join(source_folder2, filename)
            destination_file = os.path.join(destination_folder, filename)
            shutil.copyfile(source_file, destination_file)

if __name__ == "__main__":
    source_folder1 = "catalog breakdown"
    source_folder2 = "more files"
    destination_folder = "parsedData"

    copy_files(source_folder1, source_folder2, destination_folder)

In [47]:
#remove any extra space before the seperator
import os

def remove_empty_line_above(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for i in range(1, len(lines)):
        if lines[i].strip() == "####################" and lines[i - 1].strip() == "":
            lines.pop(i - 1)
            break

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def process_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            remove_empty_line_above(file_path)

if __name__ == "__main__":
    folder_path = "parsedData"
    process_files_in_folder(folder_path)

In [48]:
#remove non english content
import os
from langdetect import detect
import codecs

def is_english(text):
    try:
        lang = detect(text)
        return lang == 'en'
    except:
        return False

def process_folder(folder_path):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                with codecs.open(file_path, 'r', 'utf-8') as file_content:
                    content = file_content.read()
                    if not is_english(content):
                        print(f"Removed: {file_path} (Non-English content)")
                        file_content.close()
                        os.remove(file_path)

if __name__ == "__main__":
    input_folder = "parsedData"  # Replace with the actual folder path
    process_folder(input_folder)
    print("Processing completed.")

Removed: parsedData\191.txt (Non-English content)
Removed: parsedData\224.txt (Non-English content)
Removed: parsedData\Acreditación y Requisitos-online-courses-fall-protection-training---spanish-accreditation--requirements.txt (Non-English content)
Removed: parsedData\Acreditación y Requisitos-online-courses-persona-competente-de-osha-para-capacitacin-en-excavacin-apertura-de-zanjas-y-apuntalamiento-acreditación-y-requisitos.txt (Non-English content)
Removed: parsedData\Carlos López, Trabajador de Construcción-Testimonials.txt (Non-English content)
Removed: parsedData\Diego Enriques, Pintor-Testimonials.txt (Non-English content)
Removed: parsedData\Diego Fernando, Trabajador de obra-Testimonials.txt (Non-English content)
Removed: parsedData\GLOSARIO PERSONA COMPETENTE DE OSHA PARA CAPACITACIÓN EN EXCAVACIÓN, APERTURA DE ZANJAS Y APUNTALAMIENTO-Static.txt (Non-English content)
Removed: parsedData\Javier Sanchez Diaz, Supervisor de Obra-Testimonials.txt (Non-English content)
Removed: pa

In [49]:
#copy all course related files to a new folder
import os
import shutil

def copy_files_with_prefix(src_folder, dest_folder, prefixes):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    for filename in os.listdir(src_folder):
        for prefix in prefixes:
            if filename.startswith(prefix):
                src_path = os.path.join(src_folder, filename)
                dest_path = os.path.join(dest_folder, filename)
                shutil.copy(src_path, dest_path)
                print(f"Copied {filename} to {dest_folder}")

src_folder = "parsedData"
dest_folder = "overview n others"
prefixes = ["Overview-", "Accreditation & Requirements-", "Group Training-", "SCORM Package-", "Learning Objectives-"]

copy_files_with_prefix(src_folder, dest_folder, prefixes)

Copied Accreditation & Requirements-Arc Flash Safety Training (NFPA 70E).txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Air-IATA.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Highway.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Rail.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Vessel.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training  - Shipping Papers.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Markings.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Packaging.txt to overview n others
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Packaging_2.txt to overview n others
Copied Accreditation & Requirements-DOT H

Copied Group Training-RCRA Hazardous Waste Generator Refresher Training.txt to overview n others
Copied Group Training-RCRA Hazardous Waste Generator Training.txt to overview n others
Copied Group Training-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to overview n others
Copied Group Training-Shipping Lithium Batteries - Fully Regulated and Excepted Cells and Batteries Training (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to overview n others
Copied Group Training-Shipping Lithium Batteries - Fully Regulated Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to overview n others
Copied Learning Objectives-Arc Flash Safety Training (NFPA 70E).txt to overview n others
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Air-IATA.txt to overview n others
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Highway.txt to overview n others
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Rail.txt to overview n others


Copied SCORM Package-OSHA Lockout Tagout (LOTO) Training.txt to overview n others
Copied SCORM Package-OSHA Scaffolding Safety Training.txt to overview n others
Copied SCORM Package-OSHA Silica Awareness Training.txt to overview n others
Copied SCORM Package-OSHA Welding, Cutting, and Brazing for Construction Training.txt to overview n others
Copied SCORM Package-Pandemic Awareness, Preparedness, and Response Training.txt to overview n others
Copied SCORM Package-RCRA Hazardous Waste Generator Refresher Training.txt to overview n others
Copied SCORM Package-RCRA Hazardous Waste Generator Training.txt to overview n others
Copied SCORM Package-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to overview n others
Copied SCORM Package-Shipping Lithium Batteries - Fully Regulated and Excepted Cells and Batteries Training (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to overview n others
Copied SCORM Package-Shipping Lithium Batteries - Fully Regulated Cell

In [50]:
# #Add the new line with link for all overview n others files
# import os

# def extract_and_modify_links(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".txt"):
#             file_path = os.path.join(folder_path, filename)
#             with open(file_path, 'r+', encoding='utf-8') as file:
#                 lines = file.readlines()

#                 link = None
#                 for line in lines:
#                     if line.startswith("online-courses/") and "overview" in line:
#                         link = line.strip().replace("overview", "")
#                         break

#                 if link is not None:
#                     demo_line = ("Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n")

#                     for i, line in enumerate(lines):
#                         if "description" in line.lower():
#                             lines.insert(i, demo_line)
#                             break

#                     file.seek(0)
#                     file.writelines(lines)
#                     file.truncate()

# folder_path = "overview n others"
# extract_and_modify_links(folder_path)

In [51]:
import os

def add_demo_line(filename, lines, link):
    if "OSHA Competent Person for Excavation, Trenching, and Shoring Training" in filename or "OSHA Silica Awareness Training" in filename or "OSHA Competent Person for Fall Protection Training" in filename:
        demo_line = "Available in English and Spanish.Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n"
    else:
        demo_line = "Only available in English.Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n"
    
    for i, line in enumerate(lines):
        if "description" in line.lower():
            lines.insert(i, demo_line)
            break

def extract_and_modify_links(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r+', encoding='utf-8') as file:
            lines = file.readlines()

            link = None
            for line in lines:
                if line.startswith("online-courses/"):
                    parts = line.strip().split('/')
                    if len(parts) >= 3:
                        link = '/'.join(parts[:2]) + "/"
                    break

            if link is not None:
                add_demo_line(filename, lines, link)

                file.seek(0)
                file.writelines(lines)
                file.truncate()

folder_path = "overview n others"
extract_and_modify_links(folder_path)

In [52]:
# #Add the new line with link for all overview n others files
# import os

# def extract_and_modify_links(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".txt"):
#             file_path = os.path.join(folder_path, filename)
#             with open(file_path, 'r+', encoding='utf-8') as file:
#                 lines = file.readlines()

#                 link = None
#                 for line in lines:
#                     if line.startswith("online-courses/") and "accreditation--requirements" in line:
#                         link = line.strip().replace("accreditation--requirements", "")
#                         break

#                 if link is not None:
#                     demo_line = ("Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n")

#                     for i, line in enumerate(lines):
#                         if "description" in line.lower():
#                             lines.insert(i, demo_line)
#                             break

#                     file.seek(0)
#                     file.writelines(lines)
#                     file.truncate()

# folder_path = "overview n others"
# extract_and_modify_links(folder_path)

In [53]:
# #Add the new line with link for all overview n others files
# import os

# def extract_and_modify_links(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".txt"):
#             file_path = os.path.join(folder_path, filename)
#             with open(file_path, 'r+', encoding='utf-8') as file:
#                 lines = file.readlines()

#                 link = None
#                 for line in lines:
#                     if line.startswith("online-courses/") and "group-training" in line:
#                         link = line.strip().replace("group-training", "")
#                         break

#                 if link is not None:
#                     demo_line = ("Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n")

#                     for i, line in enumerate(lines):
#                         if "description" in line.lower():
#                             lines.insert(i, demo_line)
#                             break

#                     file.seek(0)
#                     file.writelines(lines)
#                     file.truncate()

# folder_path = "overview n others"
# extract_and_modify_links(folder_path)

In [54]:
# #Add the new line with link for all overview n others files
# import os

# def extract_and_modify_links(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".txt"):
#             file_path = os.path.join(folder_path, filename)
#             with open(file_path, 'r+', encoding='utf-8') as file:
#                 lines = file.readlines()

#                 link = None
#                 for line in lines:
#                     if line.startswith("online-courses/") and "scorm-package" in line:
#                         link = line.strip().replace("scorm-package", "")
#                         break

#                 if link is not None:
#                     demo_line = ("Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n")

#                     for i, line in enumerate(lines):
#                         if "description" in line.lower():
#                             lines.insert(i, demo_line)
#                             break

#                     file.seek(0)
#                     file.writelines(lines)
#                     file.truncate()

# folder_path = "overview n others"
# extract_and_modify_links(folder_path)

In [55]:
# #Add the new line with link for all overview n others files
# import os

# def extract_and_modify_links(folder_path):
#     for filename in os.listdir(folder_path):
#         if filename.endswith(".txt"):
#             file_path = os.path.join(folder_path, filename)
#             with open(file_path, 'r+', encoding='utf-8') as file:
#                 lines = file.readlines()

#                 link = None
#                 for line in lines:
#                     if line.startswith("online-courses/") and "learning-objectives" in line:
#                         link = line.strip().replace("learning-objectives", "")
#                         break

#                 if link is not None:
#                     demo_line = ("Do you want to know more about the course? or register/enroll/buy/purchase the course? or try the demo/trial? If you want the demo/registration/course link then go to the webpage https://hazwoper-osha.com/" + link + " \n")

#                     for i, line in enumerate(lines):
#                         if "description" in line.lower():
#                             lines.insert(i, demo_line)
#                             break

#                     file.seek(0)
#                     file.writelines(lines)
#                     file.truncate()

# folder_path = "overview n others"
# extract_and_modify_links(folder_path)

In [56]:
# #removes everything below meta_description (redundant)
# import os

# def process_file(file_path):
#     lines = []
#     inside_meta_description = False

#     with open(file_path, 'r', encoding='utf-8') as file:
#         for line in file:
#             if line.strip() == "meta_description":
#                 inside_meta_description = True
#             if not inside_meta_description:
#                 lines.append(line)
#             if line.strip() == "####################":
#                 inside_meta_description = False
#                 break  # Exit the loop if "####################" is reached

#     if inside_meta_description:  # If still inside "meta_description" block, skip processing
#         return

#     with open(file_path, 'w', encoding='utf-8') as file:
#         file.writelines(lines)

# def process_folder(folder_path):
#     for root, _, files in os.walk(folder_path):
#         for file in files:
#             if file.endswith(".txt"):
#                 file_path = os.path.join(root, file)
#                 process_file(file_path)

# if __name__ == "__main__":
#     folder_path = "parsedData"
#     process_folder(folder_path)

In [57]:
#copy all course related files and create a new folder
import os
import shutil

def copy_all_files(src_folder, dest_folder):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    for filename in os.listdir(src_folder):
        src_path = os.path.join(src_folder, filename)
        dest_path = os.path.join(dest_folder, filename)
        shutil.copy(src_path, dest_path)
        print(f"Copied {filename} to {dest_folder}")

src_folder = "overview n others"
dest_folder = "parsedData"

copy_all_files(src_folder, dest_folder)

Copied Accreditation & Requirements-Arc Flash Safety Training (NFPA 70E).txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Air-IATA.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Highway.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Rail.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Carrier Requirements - Vessel.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training  - Shipping Papers.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Markings.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Packaging.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Packaging_2.txt to parsedData
Copied Accreditation & Requirements-DOT Hazmat Function Specific Training - Placarding.txt to parsedData

Copied Group Training-RCRA Hazardous Waste Generator Training.txt to parsedData
Copied Group Training-Shipping Lithium Batteries - Excepted Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to parsedData
Copied Group Training-Shipping Lithium Batteries - Fully Regulated and Excepted Cells and Batteries Training (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to parsedData
Copied Group Training-Shipping Lithium Batteries - Fully Regulated Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to parsedData
Copied Learning Objectives-Arc Flash Safety Training (NFPA 70E).txt to parsedData
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Air-IATA.txt to parsedData
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Highway.txt to parsedData
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Rail.txt to parsedData
Copied Learning Objectives-DOT Hazmat Carrier Requirements - Vessel.txt to parsedData
Copied Learning Objectives-DOT Hazmat Function Specific Training  -

Copied Overview-Shipping Lithium Batteries - Fully Regulated and Excepted Cells and Batteries Training (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to parsedData
Copied Overview-Shipping Lithium Batteries - Fully Regulated Cells and Batteries (U.S. DOT, ICAO-IATA, IMO-IMDG).txt to parsedData
Copied SCORM Package-Arc Flash Safety Training (NFPA 70E).txt to parsedData
Copied SCORM Package-DOT Hazmat Carrier Requirements - Air-IATA.txt to parsedData
Copied SCORM Package-DOT Hazmat Carrier Requirements - Highway.txt to parsedData
Copied SCORM Package-DOT Hazmat Carrier Requirements - Rail.txt to parsedData
Copied SCORM Package-DOT Hazmat Carrier Requirements - Vessel.txt to parsedData
Copied SCORM Package-DOT Hazmat Function Specific Training  - Shipping Papers.txt to parsedData
Copied SCORM Package-DOT Hazmat Function Specific Training - Markings.txt to parsedData
Copied SCORM Package-DOT Hazmat Function Specific Training - Packaging.txt to parsedData
Copied SCORM Package-DOT Hazmat Function Speci

In [58]:
#Create a comparison file with list of all courses from overview files
import os

def extract_lines_and_append_record(folder_path, output_file):
    output_folder = "catalog breakdown"  # Name of the output folder
    output_path = os.path.join(output_folder, output_file)  # Path to the output file
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)  # Create the output folder if it doesn't exist
    
    with open(output_path, 'w', encoding='utf-8') as record:
        for filename in os.listdir(folder_path):
            if filename.startswith("Overview-") and filename.endswith(".txt"):
                file_path = os.path.join(folder_path, filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    lines = file.readlines()
                    if len(lines) >= 8:  # Ensure the file has enough lines
                        title = lines[3].strip()
                        hours = lines[5].strip()
                        price = lines[7].strip()
#                         category = lines[-4].strip()
                        category = lines[9].strip()
                        record_line = f"Title:{title} Category:{category} Duration:{hours}-Hour Price:{price}\n"
                        record.write(record_line)

folder_path = "overview n others"
output_file = "Comparison between all online courses according to price and duration.txt"
extract_lines_and_append_record(folder_path, output_file)

In [59]:
#Remove meta_description and everything below it
import os

def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    start_index = None
    end_index = None

    for i, line in enumerate(lines):
        if line.strip() == "meta_description":
            start_index = i
        elif line.strip() == "####################":
            end_index = i

    if start_index is not None and end_index is not None:
        new_lines = lines[:start_index] + lines[end_index:]
        with open(file_path, 'w', encoding='utf-8') as file:
            file.writelines(new_lines)

def process_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path)

if __name__ == "__main__":
    folder_path = "parsedData"
    process_folder(folder_path)

In [60]:
#add ### line in all files of parsedData
import os

def add_separator_line(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    if not any(line.strip() == "####################" for line in lines):
        lines.append("####################\n")

        with open(file_path, 'w', encoding='utf-8') as file:
            file.writelines(lines)

def process_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            add_separator_line(file_path)

if __name__ == "__main__":
    folder_path = "parsedData"
    process_folder(folder_path)

In [61]:
#remove an extra chunk from role of decontamination file
import os

def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    found_start = False

    for line in lines:
        if line.strip().startswith("Where Can You Learn More About Decontamination?"):
            found_start = True
        elif line.strip() == "####################":
            found_start = False
        
        if not found_start:
            new_lines.append(line)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

def main(folder_path):
    for filename in os.listdir(folder_path):
        if filename.startswith("The Role of Decontamination in Safeguarding Worker Health and Safety") and filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            process_file(file_path)
            print(f"Processed: {filename}")

if __name__ == "__main__":
    folder_path = "parsedData"
    main(folder_path)

Processed: The Role of Decontamination in Safeguarding Worker Health and Safety-Blogs.txt


In [62]:
#Copy all course files, featured course file and comparsion file to a new folder 
import os
import shutil

def copy_files_to_new_folder(source_folder, destination_folder):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)  # Create the destination folder if it doesn't exist
    
    for filename in os.listdir(source_folder):
        if filename.startswith("How many courses in") or filename == "Comparison between all online courses according to price and duration.txt" or filename=="How many featured courses are there.txt":
            source_file_path = os.path.join(source_folder, filename)
            destination_file_path = os.path.join(destination_folder, filename)
            shutil.copy(source_file_path, destination_file_path)
    
source_folder = "catalog breakdown"  # Replace with the path to your source folder
destination_folder = "Course price updates"
copy_files_to_new_folder(source_folder, destination_folder)

In [63]:
#Remove all previous courses from course files
import os
import codecs

def process_txt_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            if filename == "Comparison between all online courses according to price and duration.txt":
                continue
            if filename == "How many featured courses are there.txt":
                continue
                
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                lines = file.readlines()

            new_lines = []
            skip = False
            for line in lines:
                if "Title:" in line:
                    skip = True
                if not skip:
                    new_lines.append(line)
                
            with codecs.open(file_path, "w", encoding="utf-8") as file:
                file.writelines(new_lines)

            print(f"Processed: {filename}")

folder_path = "Course price updates"
process_txt_files(folder_path)

Processed: How many courses in DOT, IATA, IMDG Hazmat Training.txt
Processed: How many courses in DOT, IATA, IMDG Shipping Lithium Batteries.txt
Processed: How many courses in HAZWOPER (e) Training.txt
Processed: How many courses in HAZWOPER (p) Training.txt
Processed: How many courses in NFPA 70E Electrical Series.txt
Processed: How many courses in OSHA Construction Series.txt
Processed: How many courses in OSHA General Series.txt
Processed: How many courses in OSHA Outreach Training.txt
Processed: How many courses in RCRA Training.txt


In [64]:
#Update courses in all courses files using comparison file
import os

# Define the path to the folder containing the txt files
folder_path = "Course price updates"

# Read the "Comparison between all online courses according to price and duration" file
comparison_file_path = os.path.join(folder_path, "Comparison between all online courses according to price and duration.txt")
with open(comparison_file_path, "r", encoding="utf-8") as comparison_file:
    lines = comparison_file.readlines()

# Initialize a dictionary to store category-wise lines
category_lines = {}

# Process each line in the comparison file
for line in lines:
    if "Category:" in line:
        category = line.split("Category:")[1].split("Duration:")[0].strip()
        if category not in category_lines:
            category_lines[category] = []
    category_lines[category].append(line)

# Append lines to corresponding category files
for category, lines in category_lines.items():
    category_file_path = os.path.join(folder_path, f"How many courses in {category}.txt")
    with open(category_file_path, "a", encoding="utf-8") as category_file:
        for line in lines:
            category_file.write(line)

    print(f"Processed category: {category}")

print("Task completed.")

Processed category: NFPA 70E Electrical Series
Processed category: DOT, IATA, IMDG Hazmat Training
Processed category: DOT, IATA, IMDG Shipping Lithium Batteries
Processed category: OSHA Construction Series
Processed category: HAZWOPER (p) Training
Processed category: HAZWOPER (e) Training
Processed category: OSHA General Series
Processed category: OSHA Outreach Training
Processed category: RCRA Training
Task completed.


In [65]:
#add the last two lines in courses files
import os

def process_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    processed_lines = []
    exclude_processing = "Comparison between all online courses according to price and duration.txt"
    skip_file = "How many featured courses are there.txt"  # Add the filename to skip

    if os.path.basename(file_path) == skip_file:
        print(f"Skipped: {file_path}")
        return

    for line in lines:
        if exclude_processing in file_path:
            processed_lines.append(line)
            continue

        start_index = line.find("Category:")
        end_index = line.find("Duration:")

        if start_index != -1 and end_index != -1:
            processed_line = line[:start_index] + line[end_index:]
            processed_lines.append(processed_line)
        else:
            processed_lines.append(line)

    if exclude_processing not in file_path:
        processed_lines.append("To learn more, please go to the webpage https://hazwoper-osha.com/ and scroll down to see the 'Online safety training course catalog'.\n####################\n")

    with open(file_path, 'w') as file:
        file.writelines(processed_lines)

def process_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            process_file(file_path)
            print(f"Processed: {file_path}")

if __name__ == "__main__":
    folder_path = "Course price updates"  # Update this to the actual folder path
    process_folder(folder_path)

Processed: Course price updates\Comparison between all online courses according to price and duration.txt
Processed: Course price updates\How many courses in DOT, IATA, IMDG Hazmat Training.txt
Processed: Course price updates\How many courses in DOT, IATA, IMDG Shipping Lithium Batteries.txt
Processed: Course price updates\How many courses in HAZWOPER (e) Training.txt
Processed: Course price updates\How many courses in HAZWOPER (p) Training.txt
Processed: Course price updates\How many courses in NFPA 70E Electrical Series.txt
Processed: Course price updates\How many courses in OSHA Construction Series.txt
Processed: Course price updates\How many courses in OSHA General Series.txt
Processed: Course price updates\How many courses in OSHA Outreach Training.txt
Processed: Course price updates\How many courses in RCRA Training.txt
Skipped: Course price updates\How many featured courses are there.txt
Processed: Course price updates\How many featured courses are there.txt


In [66]:
#Remove the numbering from featured courses file
import re

def remove_numbering(line):
    # Use regular expression to remove numbering and dot with space
    cleaned_line = re.sub(r'^\d+\.\s*', '', line)
    return cleaned_line

def remove_numbering_from_file(file_path):
    with open(file_path, 'r+') as file:
        lines = file.readlines()
        file.seek(0)
        file.truncate(0)
        
        for line in lines:
            new_line = remove_numbering(line)
            file.write(new_line)

file_path = 'Course price updates/How many featured courses are there.txt'
remove_numbering_from_file(file_path)
print("Numbering and dots removed from the file.")

Numbering and dots removed from the file.


In [67]:
#Updates courses in featured courses file
def extract_title_duration(line):
    if line.startswith("Title:"):
        parts = line.split("Duration:")
        return parts[0].strip()
    return None

def main():
    folder_path = "Course price updates/"
    file1_name = "How many featured courses are there.txt"
    file2_name = "Comparison between all online courses according to price and duration.txt"

    with open(folder_path + file1_name, 'r') as file1:
        content1 = file1.readlines()

    with open(folder_path + file2_name, 'r') as file2:
        content2 = file2.readlines()

    updated_lines = []

    for line1 in content1:
        title_duration1 = extract_title_duration(line1)
        if title_duration1:
            matching_line = next((line2 for line2 in content2 if title_duration1 in line2), None)
            if matching_line:
                updated_lines.append(matching_line)
            else:
                updated_lines.append(line1)
        else:
            updated_lines.append(line1)

    with open(folder_path + file1_name, 'w') as file1:
        file1.writelines(updated_lines)

if __name__ == "__main__":
    main()

In [68]:
#Add catchphrases and ### line to comparison file
def add_intro_and_outro_to_output_file(output_file_path):
    intro_text = """For comparison between all online hazwoper osha courses. Duration of all online hazwoper osha courses. Price of all online hazwoper osha courses. Duration of all courses is in hours. Which/what courses are below or above a given price. Which/what courses are below or above a given duration. Which courses are around a given price. This page is used if you want to find out courses within a price or duration range. Which courses are more than given hours long. Which courses are less than given hours long. Which courses are longer than. Which courses are shorter than. How many courses are more than hours long. What courses are less than $. How many courses are less than $. How many courses are more than hours long. List of all courses that cost less than $. List of all courses that are longer than hours long. List of all courses that are shorter than hours long. Which courses cost more than. Which courses cost less than $. For comparison between all hazwoper osha courses. For comparison between all courses. Duration of all hazwoper osha courses. Price of all hazwoper osha courses. Duration of all courses is in hours. Which/what courses are below or above a given price. Which/what courses are below or above a given duration. Which courses are around a given price. This page is used if you want to find out courses within a price or duration range. Which courses are more than given hours long. Which courses are less than given hours long. Which courses are longer than. Which courses are shorter than. How many courses are more than hours long. What is the count of courses priced under $. How many courses are available for less than $ each. How many courses can you buy for under $. In the range of courses costing less than $, what's the quantity. Count the courses that fall below $ in cost. How many courses have a price tag lower than $. What's the number of courses that are priced at less than $. Tell me the quantity of courses that don't exceed $ in price. How many courses are there that you can get for less than $. Provide the total of courses that are priced under $. What's the count of courses that exceed a duration of hours. Can you tell me the number of courses that go beyond a -hour length. In how many courses does the duration extend past hours. How many courses last more than hours. Provide the quantity of courses with a duration surpassing hours. Count the courses that have a length greater than hours. What is the total of courses lasting over hours in duration. How many courses take longer than hours to complete. Give me the tally of courses exceeding a -hour duration. What's the number of courses that are of more than hours in length. Courses below or above $10. Courses below or above $5. Courses below or above $20. Courses below or above $24. Courses below or above $30. Courses below or above $40. Courses below or above $15.\n\n"""
    
    outro_text = "####################"
    
    with open(output_file_path, 'r+', encoding='utf-8') as file:
        content = file.read()
        file.seek(0, 0)
        file.write(intro_text + content + outro_text)

output_file_path = os.path.join("Course price updates", "Comparison between all online courses according to price and duration.txt")
add_intro_and_outro_to_output_file(output_file_path)

In [69]:
#Copy paste all files from Course price updates to parsedData
import os
import shutil

def copy_files(source_folder, destination_folder):
    # Get a list of all .txt files in the source folder
    txt_files = [file for file in os.listdir(source_folder) if file.endswith(".txt")]

    for txt_file in txt_files:
        source_path = os.path.join(source_folder, txt_file)
        destination_path = os.path.join(destination_folder, txt_file)

        # If the file already exists in the destination folder, replace it
        if os.path.exists(destination_path):
            os.remove(destination_path)

        # Copy the file from source to destination
        shutil.copy(source_path, destination_path)
        print(f"Copied {txt_file} to {destination_path}")

if __name__ == "__main__":
    source_folder = "Course price updates"
    destination_folder = "parsedData"

    copy_files(source_folder, destination_folder)

Copied Comparison between all online courses according to price and duration.txt to parsedData\Comparison between all online courses according to price and duration.txt
Copied How many courses in DOT, IATA, IMDG Hazmat Training.txt to parsedData\How many courses in DOT, IATA, IMDG Hazmat Training.txt
Copied How many courses in DOT, IATA, IMDG Shipping Lithium Batteries.txt to parsedData\How many courses in DOT, IATA, IMDG Shipping Lithium Batteries.txt
Copied How many courses in HAZWOPER (e) Training.txt to parsedData\How many courses in HAZWOPER (e) Training.txt
Copied How many courses in HAZWOPER (p) Training.txt to parsedData\How many courses in HAZWOPER (p) Training.txt
Copied How many courses in NFPA 70E Electrical Series.txt to parsedData\How many courses in NFPA 70E Electrical Series.txt
Copied How many courses in OSHA Construction Series.txt to parsedData\How many courses in OSHA Construction Series.txt
Copied How many courses in OSHA General Series.txt to parsedData\How many c

In [1]:
#remove the  ### line from the last
import os
import fileinput

def remove_last_line_with_marker(file_path, marker):
    lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Check if the last line is the marker and remove it if it is
    if lines and lines[-1].strip() == marker:
        lines.pop()
    
    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(lines)

def process_files_in_folder(folder_path, marker):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            remove_last_line_with_marker(file_path, marker)

if __name__ == "__main__":
    folder_path = "parsedData"  # Replace with your folder path
    marker = "####################"  # The marker line to remove
    
    process_files_in_folder(folder_path, marker)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'parsedData'

In [2]:
#remove duplicate chunks from scorm related txt files
import os

# Function to remove lines starting with a specific prefix
def remove_lines_with_prefix(file_path, prefix):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    new_lines = []
    remove_next_lines = False

    for line in lines:
        if not remove_next_lines and line.strip().startswith(prefix):
            remove_next_lines = True
        elif remove_next_lines:
            continue
        else:
            new_lines.append(line)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(new_lines)

# Function to process all files in a folder
def process_folder(folder_path, prefix):
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.txt') and file.startswith("SCORM Package-"):
                file_path = os.path.join(root, file)
                remove_lines_with_prefix(file_path, prefix)

if __name__ == "__main__":
    folder_path = "parsedData"  # Replace with the path to your folder of text files
    prefix_to_remove = "To learn more about SCORM Packages"

    process_folder(folder_path, prefix_to_remove)
    print("Lines starting with '{}' removed from files starting with 'SCORM Package-'.")

Lines starting with '{}' removed from files starting with 'SCORM Package-'.


In [76]:
#removing extra chunks from group training txt files
import os

# Define the folder path where the text files are located
folder_path = 'parsedData'

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the filename starts with "Group Training-"
    if filename.startswith("Group Training-"):
        # Construct the full path of the file
        file_path = os.path.join(folder_path, filename)
        
        # Open the file in utf-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        
        # Initialize a flag to check if the target line is found
        target_line_found = False
        
        # Create a list to store modified lines
        modified_lines = []
        
        # Iterate through the lines in the file
        for line in lines:
            if target_line_found:
                # If the target line has been found, stop processing further lines
                break
            if line.startswith("Number of expected participants."):
                # If the target line is found, set the flag to True
                target_line_found = True
            modified_lines.append(line)
        
        # Reopen the file in write mode and save the modified lines
        with open(file_path, 'w', encoding='utf-8') as file:
            file.writelines(modified_lines)

print("Processing complete.")

Processing complete.


In [3]:
#remove testimonials
import os

# Define the folder path
folder_path = 'parsedData'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Iterate through the files and remove those with "Testimonials" in the filename
for file_name in files:
    if "Testimonials" in file_name:  # Check if "Testimonials" is anywhere in the file name
        file_path = os.path.join(folder_path, file_name)
        os.remove(file_path)
        print(f"Removed: {file_name}")

print("File removal completed.")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'parsedData'

In [ ]:
# #Create embeddings
# from langchain.document_loaders import TextLoader
# from langchain.docstore.document import Document
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# import pickle
# from langchain.chat_models import ChatOpenAI
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Qdrant
# import os

# doc_list = []
# folder_path = "parsedData/"
# for filename in os.listdir(folder_path):
#     if filename.endswith('.txt') and filename != "Comparison between all online courses according to price and duration.txt":
#         file_path = os.path.join(folder_path, filename)
#         with open(file_path, 'r', encoding='utf-8') as file:
#             page_content = file.read()
#         metadata = {"source": file_path}
#         doc_list.append(Document(page_content = page_content, metadata = metadata)) 
        
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0, length_function=len, separators=['####################','-----','\n\n', '\n', ' ', '']
# )
# documents = text_splitter.split_documents(doc_list)

# file_path = "parsedData/Comparison between all online courses according to price and duration.txt"
# with open(file_path, 'r', encoding='utf-8') as file:
#     page_content = file.read()
# metadata = {"source": file_path}
# documents.insert(0,Document(page_content = page_content, metadata = metadata))

# from langchain.embeddings import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

# from langchain.vectorstores import Qdrant
# qdrant = Qdrant.from_documents(
#     documents,
#     embeddings,
#     location=":memory:",  # Local mode with in-memory storage only
#     collection_name="my_documents",
# )

# with open("Embeddings/hazwoperauto33(qdrant).pkl","wb") as f:
#     pickle.dump(qdrant,f)

In [ ]:
# #Delete all files that are of no use now
# import os
# import shutil

# # List of files/folders to keep
# files_to_keep = [
#     'parsedData',
#     'more files',
#     'catalog breakdown',
#     'course_data_category-24-8-23.json',
#     'Embeddings',
#     'Pre-process.ipynb',
#     'Embedding_process.ipynb',
#     'Chat_bot.ipynb',
#     'Data_Haz.json',
#     'New folder'
# ]

# # Function to delete all files/folders except the ones in files_to_keep list
# def delete_files_except_specific():
#     current_directory = os.getcwd()
#     for item in os.listdir(current_directory):
#         item_path = os.path.join(current_directory, item)
#         if item not in files_to_keep:
#             if os.path.isfile(item_path):
#                 os.remove(item_path)
#                 print(f"Deleted file: {item_path}")
#             elif os.path.isdir(item_path):
#                 shutil.rmtree(item_path)
#                 print(f"Deleted folder: {item_path}")

# # Calling the function to delete files/folders
# delete_files_except_specific()